# Multi-Agent Weather Satellite Analysis System

## Overview
Advanced **LLM-powered multi-agent system** for automated weather satellite imagery analysis, integrating fine-tuned vision-language models with intelligent agent orchestration to reduce manual analysis time by **60%**.

### System Architecture
The system employs **LangGraph** to orchestrate specialized agents that collaborate for comprehensive meteorological analysis:
- **Intelligent query routing** and task classification
- **Satellite image analysis** using fine-tuned vision-language models
- **Multi-source weather data** retrieval and integration
- **Historical climate analysis** and visualization
- **Interactive user interface** for real-time queries

---

## Agent Architecture & Roles

### 🎯 **Router Agent**
**Role:** Intelligent task classification and query orchestration

**Responsibilities:**
- **Satellite Analysis Detection:** Routes queries containing "satellite image," "analyze satellite," "MODIS," "Terra," or "meteorological analysis" to Satellite Agent
- **Current Weather Routing:** Directs requests with temporal indicators ("current," "now," "today," "forecast") to Weather Agent
- **Historical Analysis Classification:** Routes queries with "history," "historical," "trend," "dashboard," "visualization," or "climate" to Historical Agent
- **Temporal Awareness:** Calculates appropriate date ranges for relative time references to ensure accurate API selection

**Key Features:**
- Enhanced keyword detection with pattern matching
- Temporal analysis for intelligent routing decisions
- Dynamic date range calculation for historical queries

---

### 🛰️ **Satellite Agent** 
**Role:** Primary analysis agent for satellite imagery interpretation

**Core Implementation:** `WeatherSatelliteAnalysisAgent` class integrating fine-tuned vision-language capabilities with meteorological domain expertise

**Responsibilities:**
- **Model Integration:** Loads and manages fine-tuned **LLaVA-AdaLoRA** model through HuggingFace transformers with caching mechanisms for optimized performance
- **Prompt Engineering:** Constructs context-aware prompts via `_build_enhanced_prompt` method, integrating:
  - Temporal information
  - Geographic coordinates
  - Ground-level weather measurements
  - Satellite imagery analysis requests
- **Multimodal Processing:** Handles diverse input formats:
  - File paths
  - Base64-encoded images
  - PIL image objects

**Key Tool:** `analyze_satellite_image_tool` - Standardized interface supporting:
- Standalone imagery interpretation
- Enhanced analysis with contextual weather data
- Optional metadata integration

---

### 🌦️ **Weather Agent**
**Role:** Real-time and short-term weather data retrieval and analysis

**Core Implementation:** `IntelligentWeatherTool` with automatic API routing

**Responsibilities:**
- **Multi-API Intelligence:** `SmartWeatherAPIRouter` evaluates query dates against API capabilities:
  - **WeatherAPI:** Current and recent data (within 3 days)
  - **NASA POWER API:** Historical information beyond 3-day threshold
- **Flexible Query Processing:** Handles multiple input formats:
  - Simple city names for current weather
  - City-date combinations for specific historical requests
  - Coordinate-based queries for precise locations
- **Real-time Capabilities:** Integration with WeatherAPI provides:
  - Current weather conditions
  - 3-day forecasts
  - Recent historical data with sub-daily resolution

---

### 📊 **Historical Agent**
**Role:** Long-term climate analysis, visualization, and trend monitoring

**Responsibilities:**
- **Multi-year Data Handling:** `WeatherHistoryTool` manages large-scale data retrieval across multiple years with efficient processing strategies
- **Specialized Analysis Tools:**
  - **SPI Calculations:** Drought monitoring and severity assessment
  - **Ombrothermic Diagrams:** Climate characterization and pattern visualization
  - **Comprehensive Dashboards:** Multi-parameter visualization for climate trends
- **Performance Optimization:** 
  - Safe execution wrappers prevent token limit issues during large dataset processing
  - Bypasses LLM processing for computational tasks
  - Implements direct data manipulation strategies

---

## Key Features

### 🤖 Multi-Agent Collaboration
- **Coordinated Workflows:** Agents work together through LangGraph state management
- **Intelligent Routing:** Router Agent ensures queries reach optimal processing pathways
- **Context Sharing:** Agents exchange information for comprehensive analysis

### 🖼️ Vision-Language Integration
- **Fine-tuned Model:** Custom LLaVA-AdaLoRA model optimized for meteorological imagery
- **Multimodal Understanding:** Combines visual satellite data with textual weather information
- **Domain Expertise:** Specialized training for weather pattern recognition

### 🌐 Interactive Interface
- **Web-based Chat:** Real-time query processing through Flask interface
- **Multi-modal Inputs:** Support for text queries and image uploads
- **Contextual Responses:** Integration of satellite analysis with weather data

### 📈 Performance Optimization
- **60% Time Reduction:** Automated analysis vs manual workflows
- **Caching Mechanisms:** Optimized model loading and inference
- **Efficient API Routing:** Intelligent selection of data sources
- **Scalable Architecture:** Handles large datasets and multiple concurrent requests

---

## Technology Stack

| Component | Technology | Purpose |
|-----------|------------|---------|
| **Agent Framework** | LangGraph | Multi-agent orchestration and state management |
| **LLM** | OpenAI GPT-4 | Natural language understanding and generation |
| **Vision Model** | LLaVA-OneVision (Fine-tuned with AdaLoRA) | Satellite imagery analysis |
| **Fine-tuning** | AdaLoRA | Efficient parameter adaptation |
| **Weather APIs** | WeatherAPI, NASA POWER | Real-time and historical weather data |
| **Web Framework** | Flask | Interactive user interface |
| **Tunneling** | ngrok | Public access from Colab |
| **Deployment** | Google Colab | Cloud-based execution environment |

---

## 1. Environment Setup

### Install Required Dependencies
Installing all necessary packages for the multi-agent system:
- `langgraph`: Agent orchestration framework
- `langchain_openai`: OpenAI integration
- `flask`: Web server
- `pyngrok`: Tunneling for public access

In [ ]:
!pip install -q langgraph==0.4.3 langchain_openai langchain


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.2/151.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 19.8 MB/s eta 0:00:00


### Import Libraries
Importing all required libraries for:
- Agent orchestration (LangGraph)
- LLM integration (OpenAI)
- Web server (Flask)
- Image processing (PIL)
- Data handling (JSON, datetime)

In [ ]:

import requests
from typing import TypedDict, Optional
from langgraph.graph import StateGraph
from openai import OpenAI
import os
import inspect
from typing import Dict, List, Any, Optional, Union
from datetime import datetime, timedelta
from dataclasses import dataclass
import json
from pydantic import BaseModel, Field
from langchain.agents import AgentExecutor, Tool
from langchain.agents import create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, END
from typing import TypedDict
from langchain.chains import LLMChain
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from typing import TypedDict, List, Dict, Any, Optional
from langchain_core.messages import BaseMessage
from langchain.agents import Tool, AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

from transformers import LlavaOnevisionForConditionalGeneration, AutoProcessor
from PIL import Image
import torch
import base64
import io

import requests
from datetime import datetime, timedelta
from typing import Dict, Any, List, Optional, TypedDict, Union
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

from pydantic import BaseModel, Field
from langchain.tools import Tool, StructuredTool
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import BaseMessage
from langchain_core.prompts import MessagesPlaceholder


### API Configuration
Configure OpenAI API key for LLM access.

**Security Note**: Replace with your actual API key when running. Never commit API keys to version control.

In [ ]:
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
api_key = os.environ["OPENAI_API_KEY"]
print(api_key)

## 2. Multi-Agent System Implementation

### Weather Satellite Analysis Agent Class

This is the **core of the system** - a comprehensive multi-agent architecture that includes:

#### Agent Types:
1. **Supervisor Agent**: Orchestrates and routes tasks
2. **Weather Analysis Agent**: Analyzes satellite images
3. **Forecast Agent**: Generates weather forecasts
4. **Query Processing Agent**: Handles user queries
5. **Data Retrieval Agent**: Fetches relevant data

#### Key Components:
- **State Management**: LangGraph state tracking
- **Agent Routing**: Intelligent task distribution
- **Vision-Language Integration**: Fine-tuned LLaVA model
- **Conversation Memory**: Context-aware responses

#### Workflow:
```
User Query → Supervisor → Route to Specialist Agent → Analysis → Response
```

The system uses **LangGraph** for state management and agent coordination, ensuring:
- Proper task routing
- Agent collaboration
- State persistence
- Error handling

In [ ]:
class WeatherSatelliteAnalysisAgent:
    """Intelligent weather satellite analysis agent using fine-tuned LLaVA-AdaLoRA"""

    def __init__(self):
        self.model = None
        self.processor = None
        self.model_loaded = False
        print("Weather Satellite Analysis Agent initialized")

    def load_model(self):
        """Load the fine-tuned LLaVA-AdaLoRA model"""
        if self.model_loaded:
            return

        print("🔄 Loading LLaVA-AdaLoRA Weather Model...")

        try:
            # Load base model
            self.model = LlavaOnevisionForConditionalGeneration.from_pretrained(
                "llava-hf/llava-onevision-qwen2-7b-ov-hf",
                torch_dtype=torch.bfloat16,
                device_map="auto",
                trust_remote_code=True
            )

            self.processor = AutoProcessor.from_pretrained(
                "llava-hf/llava-onevision-qwen2-7b-ov-hf",
                trust_remote_code=True
            )

            # Load fine-tuned adapter
            self.model.load_adapter("azdin/llava-onevision-weather-adalora")

            self.model_loaded = True
            print("✅ LLaVA-AdaLoRA Weather Model loaded successfully!")

        except Exception as e:
            print(f"❌ Error loading model: {str(e)}")
            raise e

    def analyze_satellite_image(self, image_path: str, metadata: dict = None) -> str:
        """Analyze satellite image with optional metadata"""

        # Load model if not already loaded
        if not self.model_loaded:
            self.load_model()

        try:
            # Load and process image
            if isinstance(image_path, str):
                if image_path.startswith('data:image'):
                    # Handle base64 encoded images
                    image_data = image_path.split(',')[1]
                    image_bytes = base64.b64decode(image_data)
                    image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
                else:
                    # Handle file path
                    image = Image.open(image_path).convert('RGB')
            else:
                # Assume it's already a PIL Image
                image = image_path

            # Build enhanced prompt with metadata
            prompt = self._build_enhanced_prompt(metadata)

            # Prepare conversation
            conversation = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": image},
                        {"type": "text", "text": prompt}
                    ]
                }
            ]

            # Process input
            prompt_text = self.processor.apply_chat_template(
                conversation, tokenize=False, add_generation_prompt=True
            )

            inputs = self.processor(
                text=prompt_text,
                images=image,
                return_tensors="pt"
            ).to(self.model.device)

            # Generate analysis
            with torch.no_grad():
                generated_ids = self.model.generate(
                    **inputs,
                    max_new_tokens=500,
                    do_sample=False,
                    temperature=0.1,
                    pad_token_id=self.processor.tokenizer.eos_token_id
                )

            # Decode response
            generated_text = self.processor.batch_decode(
                generated_ids[:, inputs['input_ids'].shape[1]:],
                skip_special_tokens=True
            )[0]

            return generated_text.strip()

        except Exception as e:
            return f"Error analyzing satellite image: {str(e)}"

    def _build_enhanced_prompt(self, metadata: dict = None) -> str:
        """Build enhanced prompt with metadata context"""

        base_prompt = "Please analyze this MODIS Terra satellite image of Morocco and provide a comprehensive meteorological analysis."

        if not metadata:
            return base_prompt

        # Extract metadata components
        enhanced_prompt = "Please analyze this MODIS Terra satellite image and provide a comprehensive meteorological analysis using the provided context:\n\n"

        # Add date/time context
        if 'temporal_info' in metadata:
            temp_info = metadata['temporal_info']
            enhanced_prompt += f"**Temporal Context:**\n"
            enhanced_prompt += f"- Date: {temp_info.get('date_formatted', 'Unknown')}\n"
            enhanced_prompt += f"- Season: {temp_info.get('season', 'Unknown')}\n"
            enhanced_prompt += f"- Time: 12:00 noon UTC\n\n"

        # Add weather context
        if 'regional_weather' in metadata:
            enhanced_prompt += "**Ground-level Weather Data (12:00 UTC):**\n"
            for city, weather_data in metadata['regional_weather'].items():
                city_name = city.replace('_', ' ').title()
                if 'weather_12utc' in weather_data:
                    weather = weather_data['weather_12utc']
                    temp = weather.get('T2M', 'N/A')
                    humidity = weather.get('RH2M', 'N/A')
                    precip = weather.get('PRECTOTCORR', 'N/A')
                    enhanced_prompt += f"- {city_name}: {temp}°C, {humidity}% humidity, {precip}mm precipitation\n"
            enhanced_prompt += "\n"

        # Add geographic context
        if 'regional_weather' in metadata:
            enhanced_prompt += "**Geographic Context:**\n"
            for city, city_data in metadata['regional_weather'].items():
                if 'coordinates' in city_data:
                    coords = city_data['coordinates']
                    city_name = city.replace('_', ' ').title()
                    enhanced_prompt += f"- {city_name}: {coords['lat']:.2f}°N, {coords['lon']:.2f}°W\n"
            enhanced_prompt += "\n"

        enhanced_prompt += "**Analysis Requirements:**\n"
        enhanced_prompt += "1. Integrate satellite observations with ground-level measurements\n"
        enhanced_prompt += "2. Reference specific Moroccan cities and their conditions\n"
        enhanced_prompt += "3. Analyze atmospheric conditions, cloud formations, and weather patterns\n"
        enhanced_prompt += "4. Provide professional meteorological interpretation\n"
        enhanced_prompt += "5. Mention date, time (12:00 noon UTC), and seasonal context\n\n"
        enhanced_prompt += "Please provide a detailed, professional meteorological analysis."

        return enhanced_prompt


def analyze_satellite_image_tool(query: str) -> str:
    """
    Tool for satellite image analysis with metadata support.
    Format: image_path|metadata_json (metadata is optional)
    Example: /path/to/image.jpg|{"temporal_info": {"date": "2023-05-14", "season": "Spring"}}
    """
    try:
        parts = query.split('|', 1)
        image_path = parts[0].strip()

        metadata = None
        if len(parts) > 1:
            metadata_str = parts[1].strip()
            if metadata_str:
                try:
                    metadata = json.loads(metadata_str)
                except json.JSONDecodeError:
                    print("Warning: Invalid metadata JSON, proceeding without metadata")

        # Initialize the agent (this will be cached)
        if not hasattr(analyze_satellite_image_tool, 'agent'):
            analyze_satellite_image_tool.agent = WeatherSatelliteAnalysisAgent()

        # Analyze the image
        result = analyze_satellite_image_tool.agent.analyze_satellite_image(image_path, metadata)

        return json.dumps({
            "analysis": result,
            "model_used": "LLaVA-AdaLoRA Weather Specialist",
            "metadata_provided": metadata is not None
        }, indent=2)

    except Exception as e:
        return json.dumps({"error": str(e)})

# ======== Data Models ========
class WeatherParameters(BaseModel):
    """Model for NASA POWER API weather parameters."""
    T2M: Optional[float] = Field(None, description="Temperature at 2 Meters (°C)")
    T2M_MAX: Optional[float] = Field(None, description="Maximum Temperature at 2 Meters (°C)")
    T2M_MIN: Optional[float] = Field(None, description="Minimum Temperature at 2 Meters (°C)")
    PRECTOT: Optional[float] = Field(None, description="Precipitation (mm/day)")
    RH2M: Optional[float] = Field(None, description="Relative Humidity at 2 Meters (%)")
    WS10M: Optional[float] = Field(None, description="Wind Speed at 10 Meters (m/s)")
    PS: Optional[float] = Field(None, description="Surface Pressure (kPa)")
    ALLSKY_SFC_SW_DWN: Optional[float] = Field(None, description="Solar Radiation (kW-hr/m^2/day)")

class WeatherDashboardInput(BaseModel):
    """Schema for weather dashboard input"""
    location: Optional[str] = Field(None, description="The location name")
    city: Optional[str] = Field(None, description="The city coordinates")
    period: Optional[str] = Field(None, description="The date range period")
    days_count: Optional[int] = Field(None, description="Number of days")
    date: Optional[List[str]] = Field(None, description="List of dates")
    temperature: Optional[List[float]] = Field(None, description="List of average temperatures")
    temperature_max: Optional[List[float]] = Field(None, description="List of maximum temperatures")
    temperature_min: Optional[List[float]] = Field(None, description="List of minimum temperatures")
    precipitation: Optional[List[float]] = Field(None, description="List of precipitation values")
    humidity: Optional[List[float]] = Field(None, description="List of humidity values")
    wind_speed: Optional[List[float]] = Field(None, description="List of wind speed values")
    pressure: Optional[List[float]] = Field(None, description="List of pressure values")
    solar_radiation: Optional[List[float]] = Field(None, description="List of solar radiation values")

# ======== Smart API Router ========
class SmartWeatherAPIRouter:
    """Intelligent router that decides which API to use based on date requirements"""

    def __init__(self):
        self.current_date = datetime.now().date()
        self.nasa_cutoff_days = 3  # NASA API is typically 3+ days behind

    def should_use_weatherapi(self, target_date: datetime.date) -> bool:
        """Determine if WeatherAPI should be used instead of NASA API"""
        days_diff = (self.current_date - target_date).days

        # Use WeatherAPI for:
        # - Current date (today)
        # - Recent dates (within last 3 days)
        # - Future dates (forecasts)
        return days_diff <= self.nasa_cutoff_days or target_date > self.current_date

    def should_use_weatherapi_range(self, start_date: datetime.date, end_date: datetime.date) -> tuple:
        """Determine API usage for date ranges"""
        # If any date in range needs WeatherAPI, suggest splitting or using WeatherAPI
        start_needs_weather = self.should_use_weatherapi(start_date)
        end_needs_weather = self.should_use_weatherapi(end_date)

        if start_needs_weather or end_needs_weather:
            # Calculate split point
            cutoff_date = self.current_date - timedelta(days=self.nasa_cutoff_days)
            return True, cutoff_date

        return False, None

# ======== Enhanced Weather APIs ========
class WeatherAPITool:
    """Tool for current weather, recent weather, and forecasts using WeatherAPI.com"""

    def __init__(self):
        self.api_key = "weather-api-key"
        self.base_url = "http://api.weatherapi.com/v1"
        self.router = SmartWeatherAPIRouter()

    def get_current_weather(self, city: str) -> dict:
        """Get current weather information"""
        url = f"{self.base_url}/current.json?key={self.api_key}&q={city}&aqi=no"

        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            current = data['current']
            location = data['location']

            return {
                'date': datetime.now().strftime('%Y-%m-%d'),
                'location': f"{location['name']}, {location['country']}",
                'temperature': current['temp_c'],
                'temperature_max': None,  # Not available in current API
                'temperature_min': None,  # Not available in current API
                'feels_like': current['feelslike_c'],
                'precipitation': current['precip_mm'],
                'humidity': current['humidity'],
                'wind_speed': current['wind_kph'] / 3.6,  # Convert to m/s
                'wind_direction': current['wind_degree'],
                'pressure': current['pressure_mb'] / 10,  # Convert to kPa
                'condition': current['condition']['text'],
                'api_used': 'WeatherAPI'
            }
        except Exception as e:
            raise Exception(f"WeatherAPI current weather error: {str(e)}")

    def get_forecast_weather(self, city: str, days: int = 3) -> list:
        """Get forecast weather information (max 3 days)"""
        if days > 3:
            days = 3  # WeatherAPI free tier limit

        url = f"{self.base_url}/forecast.json?key={self.api_key}&q={city}&days={days}&aqi=no&alerts=no"

        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            forecast_days = data['forecast']['forecastday']
            forecast_list = []

            for day in forecast_days:
                day_data = day['day']

                forecast_info = {
                    'date': day['date'],
                    'location': f"{data['location']['name']}, {data['location']['country']}",
                    'temperature': day_data['avgtemp_c'],
                    'temperature_max': day_data['maxtemp_c'],
                    'temperature_min': day_data['mintemp_c'],
                    'precipitation': day_data['totalprecip_mm'],
                    'humidity': day_data['avghumidity'],
                    'wind_speed': day_data['maxwind_kph'] / 3.6,  # Convert to m/s
                    'condition': day_data['condition']['text'],
                    'chance_of_rain': day_data['daily_chance_of_rain'],
                    'api_used': 'WeatherAPI'
                }

                forecast_list.append(forecast_info)

            return forecast_list
        except Exception as e:
            raise Exception(f"WeatherAPI forecast error: {str(e)}")

    def get_historical_recent(self, city: str, date: str) -> dict:
        """Get historical weather for recent dates (last few days)"""
        url = f"{self.base_url}/history.json?key={self.api_key}&q={city}&dt={date}"

        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            day_data = data['forecast']['forecastday'][0]['day']
            location = data['location']

            return {
                'date': date,
                'location': f"{location['name']}, {location['country']}",
                'temperature': day_data['avgtemp_c'],
                'temperature_max': day_data['maxtemp_c'],
                'temperature_min': day_data['mintemp_c'],
                'precipitation': day_data['totalprecip_mm'],
                'humidity': day_data['avghumidity'],
                'wind_speed': day_data['maxwind_kph'] / 3.6,  # Convert to m/s
                'condition': day_data['condition']['text'],
                'api_used': 'WeatherAPI'
            }
        except Exception as e:
            raise Exception(f"WeatherAPI historical error: {str(e)}")

class IntelligentWeatherTool:
    """Intelligent weather tool that automatically chooses the right API"""

    def __init__(self):
        self.weather_api = WeatherAPITool()
        self.nasa_api = NASAPowerWeatherTool()
        self.geocoding_tool = GeocodingTool()
        self.router = SmartWeatherAPIRouter()

    def __call__(self, query: str) -> str:
        """Process weather query with intelligent API selection"""
        try:
            parts = query.split(',')

            if len(parts) == 1:
                # Simple city name query - assume current weather
                city = parts[0].strip()
                return self._get_current_weather_by_city(city)

            elif len(parts) == 2:
                # City and date/time reference
                city = parts[0].strip()
                time_ref = parts[1].strip().lower()

                if time_ref in ['current', 'now', 'today']:
                    return self._get_current_weather_by_city(city)
                elif time_ref in ['forecast', 'tomorrow', 'next days']:
                    return self._get_forecast_by_city(city)
                else:
                    # Try to parse as date
                    try:
                        date_obj = datetime.strptime(time_ref, "%Y-%m-%d")
                        return self._get_weather_by_city_and_date(city, date_obj.date())
                    except:
                        return json.dumps({"error": f"Could not parse time reference: {time_ref}"})

            elif len(parts) >= 3:
                # Coordinate-based query (lat, lon, date) - legacy support
                try:
                    latitude = float(parts[0].strip())
                    longitude = float(parts[1].strip())
                    date_str = parts[2].strip()
                    date_obj = datetime.strptime(date_str, "%Y-%m-%d").date()

                    if self.router.should_use_weatherapi(date_obj):
                        # Convert coordinates to city name for WeatherAPI
                        city_name = get_city_name(latitude, longitude)
                        if city_name:
                            return self._get_weather_by_city_and_date(city_name, date_obj)
                        else:
                            return json.dumps({"error": "Could not determine city name from coordinates"})
                    else:
                        # Use NASA API for historical data
                        return self.nasa_api(query)

                except ValueError:
                    return json.dumps({"error": "Invalid coordinate format"})

            return json.dumps({"error": "Invalid query format"})

        except Exception as e:
            return json.dumps({"error": str(e)})

    def _get_current_weather_by_city(self, city: str) -> str:
        """Get current weather for a city"""
        try:
            weather_data = self.weather_api.get_current_weather(city)
            return json.dumps(weather_data, indent=2)
        except Exception as e:
            return json.dumps({"error": str(e)})

    def _get_forecast_by_city(self, city: str) -> str:
        """Get forecast weather for a city"""
        try:
            forecast_data = self.weather_api.get_forecast_weather(city)
            return json.dumps({
                "location": forecast_data[0]['location'] if forecast_data else city,
                "forecast_days": len(forecast_data),
                "data": forecast_data
            }, indent=2)
        except Exception as e:
            return json.dumps({"error": str(e)})

    def _get_weather_by_city_and_date(self, city: str, target_date: datetime.date) -> str:
        """Get weather for a specific city and date"""
        try:
            current_date = datetime.now().date()

            if target_date == current_date:
                # Today - get current weather
                return self._get_current_weather_by_city(city)
            elif target_date > current_date:
                # Future - get forecast
                days_ahead = (target_date - current_date).days
                if days_ahead <= 3:
                    forecast_data = self.weather_api.get_forecast_weather(city, days_ahead + 1)
                    # Find the specific date
                    for day_data in forecast_data:
                        if day_data['date'] == target_date.strftime('%Y-%m-%d'):
                            return json.dumps(day_data, indent=2)
                    return json.dumps({"error": f"Forecast not available for {target_date}"})
                else:
                    return json.dumps({"error": f"Forecast only available for next 3 days. Requested date is {days_ahead} days ahead."})
            else:
                # Past date
                days_ago = (current_date - target_date).days
                if days_ago <= 3:
                    # Recent past - use WeatherAPI
                    weather_data = self.weather_api.get_historical_recent(city, target_date.strftime('%Y-%m-%d'))
                    return json.dumps(weather_data, indent=2)
                else:
                    # Older historical data - need coordinates for NASA API
                    coords_result = self.geocoding_tool(city)
                    coords_data = json.loads(coords_result)

                    if 'error' not in coords_data:
                        lat = coords_data['latitude']
                        lon = coords_data['longitude']
                        nasa_query = f"{lat},{lon},{target_date.strftime('%Y-%m-%d')}"
                        return self.nasa_api(nasa_query)
                    else:
                        return json.dumps({"error": f"Could not get coordinates for {city}"})

        except Exception as e:
            return json.dumps({"error": str(e)})

# ======== Keep Original NASA Tool (with minor updates) ========
class NASAPowerWeatherTool:
    """Tool to get weather data from NASA POWER API."""

    def __init__(self):
        self.base_url = "https://power.larc.nasa.gov/api/temporal/daily/point"
        self.default_parameters = [
            "T2M", "T2M_MAX", "T2M_MIN", "PRECTOTCORR", "RH2M",
            "WS10M", "PS", "ALLSKY_SFC_SW_DWN", "WD10M"
        ]

    def __call__(self, query: str) -> str:
        """Process a weather data query."""
        try:
            parts = query.split(',')
            if len(parts) >= 3:
                latitude = float(parts[0].strip())
                longitude = float(parts[1].strip())
                date_str = parts[2].strip()

                date_obj = datetime.strptime(date_str, "%Y-%m-%d")
                formatted_date = date_obj.strftime("%Y%m%d")

                params = {
                    "parameters": ",".join(self.default_parameters),
                    "community": "RE",
                    "longitude": longitude,
                    "latitude": latitude,
                    "start": formatted_date,
                    "end": formatted_date,
                    "format": "JSON"
                }

                response = requests.get(self.base_url, params=params)

                if response.status_code == 200:
                    data = response.json()

                    raw_parameters = {}
                    for param in self.default_parameters:
                        if param in data['properties']['parameter']:
                            param_data = data['properties']['parameter'][param]
                            raw_parameters[param] = param_data[formatted_date]

                    weather_data = {
                        "date": date_str,
                        "location": f"{latitude}°N, {longitude}°E",
                        "temperature": raw_parameters.get('T2M'),
                        "temperature_max": raw_parameters.get('T2M_MAX'),
                        "temperature_min": raw_parameters.get('T2M_MIN'),
                        "precipitation": raw_parameters.get('PRECTOTCORR'),
                        "humidity": raw_parameters.get('RH2M'),
                        "wind_speed": raw_parameters.get('WS10M'),
                        "pressure": raw_parameters.get('PS'),
                        "solar_radiation": raw_parameters.get('ALLSKY_SFC_SW_DWN'),
                        "api_used": "NASA_POWER"
                    }

                    return json.dumps(weather_data, indent=2)
                else:
                    return json.dumps({
                        "error": f"API request failed with status code {response.status_code}",
                        "message": response.text
                    })
            else:
                return json.dumps({
                    "error": "Invalid format. Please provide latitude, longitude, and date in format: lat,lon,YYYY-MM-DD"
                })

        except Exception as e:
            return json.dumps({"error": str(e)})

# ======== classes ========
class WeatherHistoryTool:
    """Tool to get historical weather data for a period."""

    def __init__(self):
        self.base_url = "https://power.larc.nasa.gov/api/temporal/daily/point"
        self.default_parameters = [
            "T2M", "T2M_MAX", "T2M_MIN", "PRECTOTCORR", "RH2M",
            "WS10M", "PS", "ALLSKY_SFC_SW_DWN","WD10M"
        ]

    def _get_year_data(self, latitude, longitude, year):
        """Get data for a specific year."""
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"

        start_date_obj = datetime.strptime(start_date, "%Y-%m-%d")
        end_date_obj = datetime.strptime(end_date, "%Y-%m-%d")

        formatted_start = start_date_obj.strftime("%Y%m%d")
        formatted_end = end_date_obj.strftime("%Y%m%d")

        params = {
            "parameters": ",".join(self.default_parameters),
            "community": "RE",
            "longitude": longitude,
            "latitude": latitude,
            "start": formatted_start,
            "end": formatted_end,
            "format": "JSON"
        }

        response = requests.get(self.base_url, params=params)

        if response.status_code == 200:
            data = response.json()
            historical_data = []

            if 'properties' in data and 'parameter' in data['properties']:
                parameters_data = data['properties']['parameter']

                current_date = start_date_obj
                while current_date <= end_date_obj:
                    date_str = current_date.strftime("%Y-%m-%d")
                    api_date_str = current_date.strftime("%Y%m%d")

                    day_data = {
                        "date": date_str,
                        "location": f"{latitude}°N, {longitude}°E"
                    }

                    field_mapping = {
                        'T2M': 'temperature',
                        'T2M_MAX': 'temperature_max',
                        'T2M_MIN': 'temperature_min',
                        'PRECTOTCORR': 'precipitation',
                        'RH2M': 'humidity',
                        'WS10M': 'wind_speed',
                        'PS': 'pressure',
                        'ALLSKY_SFC_SW_DWN': 'solar_radiation',
                        'WD10M': 'wind_direction',
                    }

                    for param, field in field_mapping.items():
                        if param in parameters_data and api_date_str in parameters_data[param]:
                            day_data[field] = parameters_data[param][api_date_str]

                    historical_data.append(day_data)
                    current_date += timedelta(days=1)

            return historical_data
        else:
            raise Exception(f"API request failed with status code {response.status_code}: {response.text}")

    def __call__(self, query: str) -> str:
        """Get weather data for a location over a date range."""
        try:
            parts = query.split(',')
            if len(parts) < 4:
                return json.dumps({
                    "error": "Invalid format. Please provide: latitude,longitude,start_date,end_date"
                })

            latitude = float(parts[0].strip())
            longitude = float(parts[1].strip())
            start_date_str = parts[2].strip()
            end_date_str = parts[3].strip()

            start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
            end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

            date_difference = (end_date - start_date).days
            if date_difference < 0:
                return json.dumps({"error": "End date must be after start date"})

            # Handle multi-year requests
            if date_difference > 365:
                # For multi-year requests, collect data year by year
                all_data = []
                start_year = start_date.year
                end_year = end_date.year

                for year in range(start_year, end_year + 1):
                    try:
                        year_data = self._get_year_data(latitude, longitude, year)
                        all_data.extend(year_data)
                    except Exception as e:
                        print(f"Error fetching data for year {year}: {str(e)}")
                        continue

                # Filter to exact date range if needed
                filtered_data = []
                for day_data in all_data:
                    day_date = datetime.strptime(day_data['date'], "%Y-%m-%d")
                    if start_date <= day_date <= end_date:
                        filtered_data.append(day_data)

                result = {
                    "location": f"{latitude}°N, {longitude}°E",
                    "period": f"{start_date_str} to {end_date_str}",
                    "days_count": len(filtered_data),
                    "data": filtered_data
                }

                return json.dumps(result, indent=2)

            else:
                # Handle single year requests (original logic)
                formatted_start = start_date.strftime("%Y%m%d")
                formatted_end = end_date.strftime("%Y%m%d")

                params = {
                    "parameters": ",".join(self.default_parameters),
                    "community": "RE",
                    "longitude": longitude,
                    "latitude": latitude,
                    "start": formatted_start,
                    "end": formatted_end,
                    "format": "JSON"
                }

                response = requests.get(self.base_url, params=params)

                if response.status_code == 200:
                    data = response.json()
                    historical_data = []

                    if 'properties' in data and 'parameter' in data['properties']:
                        parameters_data = data['properties']['parameter']

                        current_date = start_date
                        while current_date <= end_date:
                            date_str = current_date.strftime("%Y-%m-%d")
                            api_date_str = current_date.strftime("%Y%m%d")

                            day_data = {
                                "date": date_str,
                                "location": f"{latitude}°N, {longitude}°E"
                            }

                            field_mapping = {
                                'T2M': 'temperature',
                                'T2M_MAX': 'temperature_max',
                                'T2M_MIN': 'temperature_min',
                                'PRECTOTCORR': 'precipitation',
                                'RH2M': 'humidity',
                                'WS10M': 'wind_speed',
                                'PS': 'pressure',
                                'ALLSKY_SFC_SW_DWN': 'solar_radiation',
                                'WD10M': 'wind_direction',
                            }

                            for param, field in field_mapping.items():
                                if param in parameters_data and api_date_str in parameters_data[param]:
                                    day_data[field] = parameters_data[param][api_date_str]

                            historical_data.append(day_data)
                            current_date += timedelta(days=1)

                    result = {
                        "location": f"{latitude}°N, {longitude}°E",
                        "period": f"{start_date_str} to {end_date_str}",
                        "days_count": len(historical_data),
                        "data": historical_data
                    }

                    return json.dumps(result, indent=2)
                else:
                    return json.dumps({
                        "error": f"API request failed with status code {response.status_code}",
                        "message": response.text
                    })

        except Exception as e:
            return json.dumps({"error": str(e)})

class GeocodingTool:
    """Tool to convert city names to latitude and longitude coordinates."""

    def __init__(self):
        self.api_key = "openweathermap-api-key"
        self.base_url = "http://api.openweathermap.org/geo/1.0/direct"

    def __call__(self, city_name: str) -> str:
        """Get latitude and longitude coordinates for a city."""
        try:
            params = {
                "q": city_name.strip(),
                "limit": 1,
                "appid": self.api_key
            }

            response = requests.get(self.base_url, params=params)

            if response.status_code == 200:
                data = response.json()

                if data and len(data) > 0:
                    location = data[0]
                    result = {
                        "city": city_name,
                        "formatted_location": f"{location.get('name', city_name)}, {location.get('country', '')}",
                        "latitude": location["lat"],
                        "longitude": location["lon"],
                        "country": location.get("country", ""),
                        "state": location.get("state", "")
                    }
                    return json.dumps(result, indent=2)
                else:
                    return json.dumps({
                        "error": f"Could not find coordinates for city: {city_name}"
                    })
            else:
                return json.dumps({
                    "error": f"API request failed with status code {response.status_code}",
                    "message": response.text
                })

        except Exception as e:
            return json.dumps({"error": str(e)})

def get_city_name(lat, lon):
    """
    Retrieves the city name for the given latitude and longitude using OpenWeatherMap's Geocoding API.

    Parameters:
        lat (float): Latitude of the location.
        lon (float): Longitude of the location.

    Returns:
        str or None: The city name if found, otherwise None.
    """
    url = "https://api.openweathermap.org/geo/1.0/reverse"
    params = {
        "lat": lat,
        "lon": lon,
        "limit": 1,
        "appid": '97e38abedbb312d3d89c405012c76d08'
    }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # Raise an exception for HTTP errors
        data = response.json()
        if data:
            return data[0].get("name")
        else:
            print("No data found for the provided coordinates.")
            return None
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
        return None

# ======== Keep all visualization functions unchanged ========
def output_table(data: str) -> str:
    """
    Outputs a pretty table using plotly.
    data has to be a dictionary-formatted string with column names as keys and column values as values.
    """
    try:
        if isinstance(data, dict):
            data = json.dumps(data)
        data_dict = json.loads(data)
        df = pd.DataFrame(data_dict)

        fig = go.Figure(data=[go.Table(
            header={'values': list(df.columns), 'fill_color': 'lightblue', 'align': 'left'},
            cells={'values': [df[col] for col in df.columns], 'fill_color': 'lavender', 'align': 'left'}
        )])

        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        filename = f"weather_table_{timestamp}.html"
        fig.write_html(filename)

        return f"Table created successfully and saved to {filename}"
    except Exception as e:
        return f"Error creating table: {str(e)}"


# ======== Agent State ========
class AgentState(TypedDict):
    messages: List[BaseMessage]
    task: str
    current_agent: str
    result: Dict[str, Any]
    error: Optional[str]
    final_output: Optional[str]

# ======== Enhanced Multi-Agent System ========
class MultiAgentWeatherSystem:
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-4o", temperature=0.3)

        # Initialize enhanced tools
        self.intelligent_weather_tool = IntelligentWeatherTool()
        self.weather_api_tool = WeatherAPITool()
        self.history_tool = WeatherHistoryTool()
        self.geocoding_tool = GeocodingTool()
        self.router = SmartWeatherAPIRouter()

        # Create agents
        self.weather_agent = self._create_weather_agent()
        self.historical_agent = self._create_historical_agent()
        self.satellite_agent = self._create_satellite_analysis_agent()

        # Build the graph
        self.graph = self._create_graph()

    def _create_satellite_analysis_agent(self):
        """Create satellite image analysis agent"""

        tools = [
            Tool(
                name="analyze_satellite_image",
                func=analyze_satellite_image_tool,
                description=(
                    "Analyze weather satellite images using fine-tuned LLaVA-AdaLoRA model. "
                    "Format: image_path|metadata_json (metadata optional) "
                    "Provides comprehensive meteorological analysis of MODIS Terra satellite images. "
                    "Supports enhanced analysis with temporal, geographic, and weather metadata. "
                    "Example: '/path/to/morocco_satellite.jpg|{\"temporal_info\": {\"date\": \"2023-05-14\", \"season\": \"Spring\"}}'"
                )
            )
        ]

        prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a Satellite Image Analysis Agent specializing in meteorological interpretation.

            CAPABILITIES:
            - Advanced satellite image analysis using fine-tuned LLaVA-AdaLoRA model
            - Professional meteorological interpretation
            - Integration of satellite observations with ground-level data
            - Support for temporal and geographic metadata enhancement

            USAGE PATTERNS:
            - "analyze satellite image [path]" → Basic satellite analysis
            - "analyze [image] with metadata [json]" → Enhanced analysis with context
            - "weather analysis of satellite image [path]" → Meteorological focus

            METADATA SUPPORT:
            You can provide enhanced analysis by including:
            - Temporal context (date, season, time)
            - Ground-level weather measurements
            - Geographic coordinates and city information

            INSTRUCTIONS:
            - Always use the analyze_satellite_image tool for image analysis
            - Format: image_path|metadata_json (metadata optional)
            - Provide professional meteorological interpretation
            - Reference specific locations and weather conditions when metadata available
            - Explain analysis confidence and model capabilities
            """),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ])

        agent = create_openai_tools_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, tools=tools, verbose=True)

    def run_satellite_agent(self, state: AgentState) -> AgentState:
        """Execute the satellite analysis agent"""
        try:
            result = self.satellite_agent.invoke({"input": state["task"]})
            state["result"] = {
                "status": "success",
                "agent": "Weather Satellite Analysis Agent",
                "output": result["output"]
            }
        except Exception as e:
            state["error"] = str(e)
            state["current_agent"] = "error"

        return state

    def _create_weather_agent(self):
        """Create the enhanced weather data agent with intelligent API routing"""
        current_date = datetime.now().strftime('%Y-%m-%d')
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        tools = [
            Tool(
                name="get_intelligent_weather",
                func=self.intelligent_weather_tool,
                description=(
                    "Smart weather tool that automatically chooses the best API based on date requirements. "
                    "Supports multiple formats:\n"
                    "- City name only: 'London' (gets current weather)\n"
                    "- City + time reference: 'Paris,current' or 'Tokyo,forecast' or 'Berlin,2025-05-27'\n"
                    "- Coordinates: 'lat,lon,date' (for historical data)\n"
                    "Automatically handles:\n"
                    "- Current weather (today)\n"
                    "- Recent weather (last 3 days) using WeatherAPI\n"
                    "- Forecasts (next 3 days) using WeatherAPI\n"
                    "- Historical data (older than 3 days) using NASA API"
                )
            ),
            Tool(
                name="get_current_weather",
                func=lambda city: json.dumps(self.weather_api_tool.get_current_weather(city), indent=2),
                description="Get current weather for a city using WeatherAPI (real-time data)"
            ),
            Tool(
                name="get_forecast_weather",
                func=lambda city: json.dumps({
                    "location": city,
                    "forecast_days": 3,
                    "data": self.weather_api_tool.get_forecast_weather(city)
                }, indent=2),
                description="Get 3-day weather forecast for a city using WeatherAPI"
            ),
            Tool(
                name="get_city_coordinates",
                func=self.geocoding_tool,
                description="Convert a city name to latitude and longitude coordinates"
            )
        ]

        prompt = ChatPromptTemplate.from_messages([
            ("system", f"""Today's date is {current_date} (current time: {current_time}).

            You are an Enhanced Weather Data Retrieval Agent with intelligent API routing capabilities.

            CAPABILITIES:
            - Current weather (today): Real-time data via WeatherAPI
            - Recent weather (last 3 days): Historical data via WeatherAPI
            - Weather forecasts (next 3 days): Forecast data via WeatherAPI
            - Historical weather (older than 3 days): NASA POWER API

            TEMPORAL INTELLIGENCE:
            - "current weather" → use today's date: {current_date}
            - "yesterday" → use {(datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')}
            - "tomorrow" → use forecast for {(datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')}
            - "last week", "recent weather" → calculate appropriate recent dates
            - "forecast", "next few days" → use forecast data

            SMART TOOL SELECTION:
            1. For current/today queries: Use get_current_weather or get_intelligent_weather
            2. For forecast queries: Use get_forecast_weather
            3. For flexible queries: Use get_intelligent_weather (it auto-detects best API)
            4. For specific dates: Use get_intelligent_weather with city,date format

            IMPORTANT:
            - WeatherAPI provides real-time current weather and forecasts
            - NASA API is used for historical data older than 3 days
            - Always inform users about forecast limitations (max 3 days ahead)

            Always return results in a structured, user-friendly format."""),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ])

        agent = create_openai_tools_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, tools=tools, verbose=True)

    def _create_detailed_table_safely(self, query: str) -> str:
        """Create detailed table from full dataset without passing through LLM"""
        try:
            # Get the full data directly
            weather_data_json = self.history_tool(query)
            data_dict = json.loads(weather_data_json)

            # Add city name if possible
            parts = query.split(',')
            if len(parts) >= 2:
                try:
                    latitude = float(parts[0].strip())
                    longitude = float(parts[1].strip())
                    city_name = get_city_name(latitude, longitude)
                    if city_name:
                        data_dict['city'] = city_name
                except:
                    pass

            if 'data' in data_dict and data_dict['data']:
                # Convert to table format
                table_data = {
                    'Date': [],
                    'Temperature (°C)': [],
                    'Max Temp (°C)': [],
                    'Min Temp (°C)': [],
                    'Precipitation (mm)': [],
                    'Humidity (%)': [],
                    'Wind Speed (m/s)': [],
                    'Pressure (kPa)': []
                }

                for day in data_dict['data']:
                    table_data['Date'].append(day.get('date', ''))
                    table_data['Temperature (°C)'].append(day.get('temperature', ''))
                    table_data['Max Temp (°C)'].append(day.get('temperature_max', ''))
                    table_data['Min Temp (°C)'].append(day.get('temperature_min', ''))
                    table_data['Precipitation (mm)'].append(day.get('precipitation', ''))
                    table_data['Humidity (%)'].append(day.get('humidity', ''))
                    table_data['Wind Speed (m/s)'].append(day.get('wind_speed', ''))
                    table_data['Pressure (kPa)'].append(day.get('pressure', ''))

                return output_table(json.dumps(table_data))
            else:
                return "Error: No weather data available for table creation"

        except Exception as e:
            return f"Error creating detailed table: {str(e)}"

    def _get_historical_weather_display(self, query: str) -> str:
        """Get full historical weather data and save to file to avoid token limits"""
        try:
            # Get the full data directly
            weather_data_json = self.history_tool(query)
            data_dict = json.loads(weather_data_json)

            # Add city name if possible
            parts = query.split(',')
            if len(parts) >= 2:
                try:
                    latitude = float(parts[0].strip())
                    longitude = float(parts[1].strip())
                    city_name = get_city_name(latitude, longitude)
                    if city_name:
                        data_dict['city'] = city_name
                except:
                    pass

            # Save the complete dataset to a JSON file
            timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
            filename = f"historical_weather_data_{timestamp}.json"

            with open(filename, 'w') as f:
                json.dump(data_dict, f, indent=2)

            # Return only a summary message (like visualization functions do)
            location = data_dict.get('city', data_dict.get('location', 'Unknown'))
            period = data_dict.get('period', '')
            total_days = data_dict.get('days_count', len(data_dict.get('data', [])))

            return f"Historical weather data retrieved successfully for {location} ({period}). Complete dataset with {total_days} daily records saved to {filename}"

        except Exception as e:
            return f"Error retrieving historical weather data: {str(e)}"


    def _get_historical_weather_safely(self, query: str) -> str:
        """Get historical weather data without passing large datasets through LLM"""
        try:
            parts = query.split(',')
            if len(parts) < 4:
                return "Error: Invalid format. Please provide: latitude,longitude,start_date,end_date"

            latitude = float(parts[0].strip())
            longitude = float(parts[1].strip())
            start_date_str = parts[2].strip()
            end_date_str = parts[3].strip()

            # Calculate date range
            start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
            end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
            days_diff = (end_date - start_date).days

            # Get the data directly without going through LLM
            weather_data_json = self.history_tool(query)

            # Parse and add city name if possible
            try:
                data_dict = json.loads(weather_data_json)

                # Try to get city name from coordinates
                city_name = get_city_name(latitude, longitude)
                if city_name:
                    data_dict['city'] = city_name

                # Create summary for LLM instead of full data
                if 'data' in data_dict and len(data_dict['data']) > 0:
                    summary = {
                        "location": data_dict.get('location', 'Unknown'),
                        "city": city_name if city_name else data_dict.get('location', 'Unknown'),
                        "period": data_dict.get('period', ''),
                        "days_count": data_dict.get('days_count', 0),
                        "data_summary": {
                            "total_days": len(data_dict['data']),
                            "date_range": f"{data_dict['data'][0]['date']} to {data_dict['data'][-1]['date']}",
                            "avg_temperature": round(np.mean([d.get('temperature', 0) for d in data_dict['data'] if d.get('temperature') is not None]), 2),
                            "total_precipitation": round(sum([d.get('precipitation', 0) for d in data_dict['data'] if d.get('precipitation') is not None]), 2),
                            "avg_humidity": round(np.mean([d.get('humidity', 0) for d in data_dict['data'] if d.get('humidity') is not None]), 2)
                        },
                        "visualization_available": True,
                        "full_data_size": f"{len(data_dict['data'])} daily records"
                    }
                    return json.dumps(summary, indent=2)
                else:
                    return weather_data_json

            except Exception as e:
                return f"Data retrieved successfully but summary creation failed: {str(e)}"

        except Exception as e:
            return f"Error getting historical weather data: {str(e)}"

    def _create_spi_safely(self, query: str) -> str:
        """Create SPI analysis without passing large data through LLM"""
        try:
            parts = query.split(',')
            if len(parts) < 4:
                return "Error: Invalid format. Please provide: latitude,longitude,start_date,end_date"

            latitude = float(parts[0].strip())
            longitude = float(parts[1].strip())
            start_date_str = parts[2].strip()
            end_date_str = parts[3].strip()

            # Validate that we have enough data for SPI (at least 12 months recommended)
            start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
            end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
            months_diff = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)

            if months_diff < 12:
                return "Warning: SPI analysis works best with at least 12+ months of data for meaningful results. Consider extending your date range."

            # Get the data directly without going through LLM
            weather_data_json = self.history_tool(query)

            # Parse and add city name if possible
            try:
                data_dict = json.loads(weather_data_json)

                # Try to get city name from coordinates
                city_name = get_city_name(latitude, longitude)
                if city_name:
                    data_dict['city'] = city_name

                weather_data_json = json.dumps(data_dict)
            except:
                pass  # If parsing fails, use original data

            # Process the data for SPI analysis
            return create_spi_visualization(weather_data_json)

        except Exception as e:
            return f"Error creating SPI analysis: {str(e)}"

    def _create_dashboard_safely(self, query: str) -> str:
        """Create dashboard without passing large data through LLM"""
        try:
            # Get the data directly
            weather_data_json = self.history_tool(query)

            # Parse and add city name if possible
            try:
                data_dict = json.loads(weather_data_json)

                # Try to get city name from coordinates if query has coordinates
                parts = query.split(',')
                if len(parts) >= 2:
                    try:
                        latitude = float(parts[0].strip())
                        longitude = float(parts[1].strip())
                        city_name = get_city_name(latitude, longitude)
                        if city_name:
                            data_dict['city'] = city_name
                    except:
                        pass

                weather_data_json = json.dumps(data_dict)
            except:
                pass

            # Create dashboard directly
            return create_weather_dashboard(**json.loads(weather_data_json))

        except Exception as e:
            return f"Error creating dashboard: {str(e)}"

    def _create_historical_agent(self):
        """Create the historical weather agent with enhanced capabilities"""

        # Create structured tool for dashboard
        create_weather_dashboard_tool = StructuredTool.from_function(
            name="create_weather_dashboard",
            func=create_weather_dashboard,
            description="Create comprehensive weather dashboard visualization.",
            args_schema=WeatherDashboardInput
        )

        current_date = datetime.now().strftime('%Y-%m-%d')
        current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        tools = [
            Tool(
                name="get_historical_weather",
                func=self._get_historical_weather_display,  # For display purposes
                description=(
                    "Get and display historical weather data for a location over a date range. "
                    "Shows sample data and summary statistics without token limits. "
                    "Format: latitude,longitude,start_date,end_date"
                )
            ),
            Tool(
                name="get_recent_weather_range",
                func=self._get_recent_weather_range,
                description=(
                    "Get recent weather data (last few days) using WeatherAPI. "
                    "Format: city_name,days_back "
                    "Example: London,3 (gets last 3 days of weather for London)"
                )
            ),
            Tool(
                name="get_city_coordinates",
                func=self.geocoding_tool,
                description="Convert a city name to latitude and longitude coordinates."
            ),
            Tool(
                name="create_spi_analysis",
                func=self._create_spi_safely,
                description=(
                    "Create Standardized Precipitation Index (SPI) analysis for drought monitoring. "
                    "SPI quantifies precipitation deficit/surplus over 3, 6, and 12-month periods. "
                    "Format: latitude,longitude,start_date,end_date "
                    "Requires at least 12+ months of data for meaningful analysis."
                )
            ),
            Tool(
                name="create_ombrothermic_diagram",
                func=self._create_ombrothermic_safely,
                description=(
                    "Create an ombrothermic (climate) diagram showing temperature and precipitation relationship. "
                    "Handles large datasets efficiently. Input: city coordinates and date range."
                    "Format: latitude,longitude,start_date,end_date"
                )
            ),
            Tool(
                name="create_detailed_table",
                func=self._create_detailed_table_safely,  # For detailed tables
                description=(
                    "Create detailed daily weather table from full historical dataset. "
                    "Handles large datasets efficiently. "
                    "Format: latitude,longitude,start_date,end_date"
                )
            ),
            Tool(
                name="create_dashboard_from_coordinates",
                func=self._create_dashboard_safely,
                description=(
                    "Create weather dashboard from coordinate-based historical data. "
                    "Handles large datasets efficiently without token limits. "
                    "Format: latitude,longitude,start_date,end_date"
                )
            )
        ]

        prompt = ChatPromptTemplate.from_messages([
            ("system", f"""Today's date is {current_date} (current time: {current_time}).

            You are a Historical Weather Agent with enhanced multi-API capabilities and visualization tools.

            API SELECTION INTELLIGENCE:
            - For recent historical data (last 3 days): Use get_recent_weather_range with WeatherAPI
            - For older historical data (4+ days ago): Use get_historical_weather with NASA POWER API
            - For date ranges spanning recent and historical: Combine both APIs appropriately

            TEMPORAL AWARENESS:
            - "recent trends", "last week" → calculate dates from today: {current_date}
            - "last 3 days" → use get_recent_weather_range
            - "last month" → use from {(datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')} to {current_date}
            - "past year" → use historical data with NASA API

            CAPABILITIES:
            1. Multi-year historical analysis using NASA POWER API
            2. Recent weather trends using WeatherAPI
            3. Climate visualizations (ombrothermic diagrams)
            4. Drought analysis (SPI calculations)
            5. Comprehensive dashboards and tables

            IMPORTANT WORKFLOWS:
            - For historical data display: Use get_historical_weather (shows sample + summary)
            - For detailed tables: Use create_detailed_table (processes full dataset)
            - For ombrothermic diagrams: Get coordinates → use create_ombrothermic_diagram directly
            - For SPI analysis: Ensure 12+ months of data → use create_spi_analysis
            - For dashboards from coordinates: Use create_dashboard_from_coordinates for large datasets

            TABLE CREATION:
            - When users request "table" or "detailed table": Use create_detailed_table
            - This processes the full daily dataset into a proper table format

            Always choose the most appropriate API based on the date requirements and inform users
            about data sources and limitations."""),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ])

        agent = create_openai_tools_agent(self.llm, tools, prompt)
        return AgentExecutor(agent=agent, tools=tools, verbose=True)

    def _get_recent_weather_range(self, query: str) -> str:
        """Get recent weather data for multiple days using WeatherAPI"""
        try:
            parts = query.split(',')
            if len(parts) != 2:
                return json.dumps({"error": "Format: city_name,days_back (e.g., London,3)"})

            city = parts[0].strip()
            days_back = int(parts[1].strip())

            if days_back > 7:  # WeatherAPI limitation
                return json.dumps({"error": "WeatherAPI supports maximum 7 days of historical data"})

            weather_data = []
            current_date = datetime.now().date()

            for i in range(days_back):
                target_date = current_date - timedelta(days=i+1)
                try:
                    day_weather = self.weather_api_tool.get_historical_recent(city, target_date.strftime('%Y-%m-%d'))
                    weather_data.insert(0, day_weather)  # Insert at beginning to maintain chronological order
                except Exception as e:
                    print(f"Error getting weather for {target_date}: {str(e)}")
                    continue

            result = {
                "location": weather_data[0]['location'] if weather_data else city,
                "period": f"Last {days_back} days",
                "days_count": len(weather_data),
                "data": weather_data,
                "api_used": "WeatherAPI"
            }

            return json.dumps(result, indent=2)

        except Exception as e:
            return json.dumps({"error": str(e)})

    def _create_ombrothermic_safely(self, query: str) -> str:
        """Create ombrothermic diagram without passing large data through LLM"""
        try:
            parts = query.split(',')
            if len(parts) < 4:
                return "Error: Invalid format. Please provide: latitude,longitude,start_date,end_date"

            latitude = float(parts[0].strip())
            longitude = float(parts[1].strip())
            start_date_str = parts[2].strip()
            end_date_str = parts[3].strip()

            # Get the data directly without going through LLM
            weather_data_json = self.history_tool(f"{latitude},{longitude},{start_date_str},{end_date_str}")

            # Process the data for ombrothermic diagram
            return create_ombrothermic_diagram(weather_data_json)

        except Exception as e:
            return f"Error creating ombrothermic diagram: {str(e)}"

    def _create_graph(self):
        """Define the enhanced graph structure"""
        workflow = StateGraph(AgentState)

        # Add nodes
        workflow.add_node("router", self.route_task)
        workflow.add_node("weather_agent", self.run_weather_agent)
        workflow.add_node("historical_agent", self.run_historical_agent)
        workflow.add_node("satellite_agent", self.run_satellite_agent)
        workflow.add_node("finalizer", self.finalize_result)
        workflow.add_node("error_handler", self.handle_error)


        # Set entry point
        workflow.set_entry_point("router")

        # Add conditional edges from router
        workflow.add_conditional_edges(
            "router",
            lambda state: state["current_agent"],
            {
                "weather": "weather_agent",
                "historical": "historical_agent",
                "satellite": "satellite_agent",
                "error": "error_handler"
            }
        )

        # From agents to finalizer
        workflow.add_edge("weather_agent", "finalizer")
        workflow.add_edge("historical_agent", "finalizer")
        workflow.add_edge("satellite_agent", "finalizer")

        # End states
        workflow.add_edge("finalizer", END)
        workflow.add_edge("error_handler", END)

        return workflow.compile()

    def route_task(self, state: AgentState) -> AgentState:
        """Enhanced routing with better temporal and API awareness"""
        task = state["task"]
        task_lower = task.lower()

        # Enhanced keywords for routing
        current_weather_keywords = [
            "current", "now", "today", "right now", "present", "at the moment"
        ]

        forecast_keywords = [
            "forecast", "tomorrow", "next", "coming", "future", "will be", "going to be"
        ]

        recent_keywords = [
            "yesterday", "last night", "last few days", "recent", "past few days"
        ]

        # SATELLITE KEYWORDS (check FIRST)
        satellite_keywords = [
            "satellite image", "analyze satellite", "modis", "terra",
            "satellite analysis", "image analysis", "meteorological analysis",
            "weather image", "analyze image"
        ]

        historical_keywords = [
            "history", "historical", "range", "from", "to", "between", "dates",
            "trend", "over time", "past", "table", "dashboard", "visualization",
            "ombrothermic", "ombrothermique", "climate", "diagram", "climatic",
            "spi", "drought", "last month", "last year", "past year"
        ]

        # Smart routing logic - CHECK SATELLITE FIRST
        if any(keyword in task_lower for keyword in satellite_keywords):
            # Satellite analysis goes to satellite agent
            state["current_agent"] = "satellite"
        elif any(keyword in task_lower for keyword in current_weather_keywords + forecast_keywords + recent_keywords):
            # Current, forecast, or recent weather queries go to weather agent
            state["current_agent"] = "weather"
        elif any(keyword in task_lower for keyword in historical_keywords):
            # Historical analysis goes to historical agent
            state["current_agent"] = "historical"
        else:
            # Default to weather agent for simple queries
            state["current_agent"] = "weather"

        print(f"\n[Enhanced Router] Routing task to {state['current_agent']} agent")
        print(f"[Enhanced Router] Task type detected: {task_lower}")
        return state

    def run_weather_agent(self, state: AgentState) -> AgentState:
        """Execute the enhanced weather agent"""
        try:
            result = self.weather_agent.invoke({"input": state["task"]})
            state["result"] = {
                "status": "success",
                "agent": "Enhanced Weather Data Agent",
                "output": result["output"]
            }
        except Exception as e:
            state["error"] = str(e)
            state["current_agent"] = "error"

        return state

    def run_historical_agent(self, state: AgentState) -> AgentState:
        """Execute the enhanced historical weather agent"""
        try:
            result = self.historical_agent.invoke({"input": state["task"]})
            state["result"] = {
                "status": "success",
                "agent": "Enhanced Historical Weather Agent",
                "output": result["output"]
            }
        except Exception as e:
            state["error"] = str(e)
            state["current_agent"] = "error"

        return state

    def finalize_result(self, state: AgentState) -> AgentState:
        """Finalize the successful result"""
        try:
            if "result" in state and isinstance(state["result"], dict):
                output = state["result"].get("output", "Task completed successfully")
                state["final_output"] = output
            else:
                state["final_output"] = "Task completed successfully"

            print("\n[Finalizer] Task completed successfully")
        except Exception as e:
            state["final_output"] = f"Error in finalization: {str(e)}"

        return state

    def handle_error(self, state: AgentState) -> AgentState:
        """Handle errors in the workflow"""
        error_msg = state.get("error", "Unknown error occurred")
        state["final_output"] = f"Error: {error_msg}"
        print(f"\n[Error Handler] {error_msg}")
        return state

    def run(self, task: str) -> Dict[str, Any]:
        """Execute a task through the enhanced multi-agent system"""
        initial_state: AgentState = {
            "messages": [],
            "task": task,
            "current_agent": "",
            "result": {},
            "error": None,
            "final_output": None
        }

        print(f"[Enhanced System] Starting with task: {task}")

        try:
            final_state = self.graph.invoke(initial_state)
        except Exception as e:
            print(f"[Enhanced System] Graph execution error: {str(e)}")
            final_state = initial_state
            final_state["error"] = str(e)

        return {
            "task": task,
            "status": "success" if final_state.get("final_output") and not final_state.get("error") else "failed",
            "output": final_state.get("final_output", "No output generated"),
            "error": final_state.get("error")
        }

# ======== Add all the missing visualization functions ========
def create_ombrothermic_diagram(data: str) -> str:
    """
    Create an ombrothermic diagram (Diagramme ombrothermique) for climate analysis.
    Shows relationship between temperature and precipitation following Gaussen's index.
    """
    try:
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots
        import numpy as np
        from datetime import datetime

        # Parse input data
        if isinstance(data, str):
            data_dict = json.loads(data)
        else:
            data_dict = data

        # Extract monthly data
        months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

        # Initialize monthly aggregators
        monthly_temp_max = {month: [] for month in range(1, 13)}
        monthly_temp_min = {month: [] for month in range(1, 13)}
        monthly_precip = {month: [] for month in range(1, 13)}

        # Process data
        if 'data' in data_dict and isinstance(data_dict['data'], list):
            for day_data in data_dict['data']:
                date_str = day_data.get('date', '')
                if date_str:
                    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
                    month = date_obj.month

                    # Collect temperature data
                    if 'temperature_max' in day_data and day_data['temperature_max'] is not None:
                        monthly_temp_max[month].append(day_data['temperature_max'])
                    if 'temperature_min' in day_data and day_data['temperature_min'] is not None:
                        monthly_temp_min[month].append(day_data['temperature_min'])
                    if 'precipitation' in day_data and day_data['precipitation'] is not None:
                        monthly_precip[month].append(day_data['precipitation'])

        # Calculate monthly averages
        avg_temp_max = []
        avg_temp_min = []
        avg_precip = []

        for month in range(1, 13):
            # Temperature averages
            if monthly_temp_max[month]:
                avg_temp_max.append(np.mean(monthly_temp_max[month]))
            else:
                avg_temp_max.append(None)

            if monthly_temp_min[month]:
                avg_temp_min.append(np.mean(monthly_temp_min[month]))
            else:
                avg_temp_min.append(None)

            # Precipitation: sum for the month then divide by number of years
            if monthly_precip[month]:
                # Calculate total days in dataset to determine number of years
                total_days = sum(len(v) for v in monthly_precip.values())
                approx_years = total_days / 365
                monthly_total = sum(monthly_precip[month])
                avg_precip.append(monthly_total / max(1, approx_years))
            else:
                avg_precip.append(None)

        # Create figure with secondary y-axis
        fig = make_subplots(
            specs=[[{"secondary_y": True}]],
            subplot_titles=[""]
        )

        # Add precipitation bars
        fig.add_trace(
            go.Bar(
                x=months,
                y=avg_precip,
                name='Précipitations',
                marker_color='lightblue',
                yaxis='y',
                opacity=0.7
            ),
            secondary_y=False
        )

        # Add temperature lines
        if any(avg_temp_max):
            fig.add_trace(
                go.Scatter(
                    x=months,
                    y=avg_temp_max,
                    mode='lines+markers',
                    name='Température maximale',
                    line=dict(color='red', width=3),
                    marker=dict(size=8)
                ),
                secondary_y=True
            )

        if any(avg_temp_min):
            fig.add_trace(
                go.Scatter(
                    x=months,
                    y=avg_temp_min,
                    mode='lines+markers',
                    name='Température minimale',
                    line=dict(color='blue', width=3),
                    marker=dict(size=8)
                ),
                secondary_y=True
            )

        # Update axes
        fig.update_xaxes(title_text="Mois")
        fig.update_yaxes(title_text="Précipitations (mm)", secondary_y=False)
        fig.update_yaxes(title_text="Température (°C)", secondary_y=True)

        # Update layout
        location = data_dict.get('location', 'Unknown Location')
        city = data_dict.get('city', location)

        # If city is still coordinates, try to get real city name
        if '°N' in str(city) and '°E' in str(city):
            try:
                # Extract coordinates from location string like "34.022405°N, -6.834543°E"
                coords = location.replace('°N', '').replace('°E', '').split(', ')
                if len(coords) == 2:
                    lat = float(coords[0])
                    lon = float(coords[1])
                    city_name = get_city_name(lat, lon)
                    if city_name:
                        city = city_name
            except:
                pass  # If anything fails, keep the original city value
        period = data_dict.get('period', '')

        fig.update_layout(
            title={
                'text': f"Diagramme Ombrothermique - {city}<br><sup>{period}</sup>",
                'x': 0.5,
                'xanchor': 'center',
                'font': {'size': 20}
            },
            hovermode='x unified',
            plot_bgcolor='white',
            height=600,
            legend=dict(
                orientation="h",
                yanchor="bottom",
                y=1.02,
                xanchor="right",
                x=1
            )
        )

        # Add grid
        fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')
        fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgray')

        # Save the figure
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        filename = f"ombrothermic_diagram_{timestamp}.html"
        fig.write_html(filename)

        return f"Ombrothermic diagram created successfully and saved to {filename}"

    except Exception as e:
        return f"Error creating ombrothermic diagram: {str(e)}"

def create_weather_dashboard(**kwargs) -> str:
    """Create comprehensive weather dashboard with improved styling"""
    try:
        data_dict = kwargs

        # Handle different data formats
        if "data" in data_dict and isinstance(data_dict["data"], list):
            converted_data = {
                "location": data_dict.get("location", "Unknown"),
                "city": data_dict.get("city", "Unknown")
            }

            if data_dict["data"]:
                sample_point = data_dict["data"][0]
                for key in sample_point.keys():
                    if key == "location":
                        continue
                    converted_data[key] = [point.get(key) for point in data_dict["data"]]

            data_dict = converted_data

        # Fix location if it's an array
        if isinstance(data_dict.get('location'), list):
            locations = data_dict['location']
            if all(loc == locations[0] for loc in locations):
                data_dict['location'] = locations[0]

        # Validate required data
        if 'date' not in data_dict:
            return f"Error: Weather data must include 'date' field. Got keys: {list(data_dict.keys())}"

        # Create subplots
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                '<b>Temperature Trends</b>',
                '<b>Precipitation</b>',
                '<b>Humidity & Pressure</b>',
                '<b>Wind Conditions</b>'
            ),
            specs=[[{"secondary_y": False}, {"secondary_y": False}],
                   [{"secondary_y": True}, {"type": "polar"}]],
            vertical_spacing=0.15,
            horizontal_spacing=0.08
        )

        dates = data_dict.get('date', [])

        # Temperature subplot
        if 'temperature' in data_dict:
            fig.add_trace(
                go.Scatter(
                    x=dates,
                    y=data_dict['temperature'],
                    name='Avg Temp',
                    line=dict(color='#FF6B6B', width=3),
                    mode='lines+markers',
                    marker=dict(size=6)
                ),
                row=1, col=1
            )

        # Calculate and add Feels Like Temperature
        if 'temperature' in data_dict and 'humidity' in data_dict:
            # Simple feels like calculation: adjusting temperature based on humidity
            feels_like = []
            for temp, humidity in zip(data_dict['temperature'], data_dict['humidity']):
                if temp is not None and humidity is not None:
                    # More sophisticated feels like calculation
                    # When humidity is high, temperature feels warmer in hot weather and cooler in cold weather
                    if temp >= 27:  # Hot weather
                        heat_index = temp + (0.5 * (temp + 61.0 + (temp - 68.0) * 1.2 + humidity * 0.094))
                        feels_like_temp = 0.5 * (temp + heat_index)
                    else:  # Cooler weather
                        feels_like_temp = temp + (0.2 * (humidity - 50))
                    feels_like.append(feels_like_temp)
                else:
                    feels_like.append(None)

            fig.add_trace(
                go.Scatter(
                    x=dates,
                    y=feels_like,
                    name='Feels Like',
                    line=dict(color='#FFB84D', width=2, dash='dot'),
                    mode='lines+markers',
                    marker=dict(size=4)
                ),
                row=1, col=1
            )

        # Precipitation subplot
        if 'precipitation' in data_dict:
            fig.add_trace(
                go.Bar(
                    x=dates,
                    y=data_dict['precipitation'],
                    name='Precipitation',
                    marker=dict(
                        color=data_dict['precipitation'],
                        colorscale='Blues',
                        showscale=False,
                        line=dict(color='#1E6BA8', width=1)
                    )
                ),
                row=1, col=2
            )

        # Humidity and Pressure subplot
        if 'humidity' in data_dict:
            fig.add_trace(
                go.Scatter(
                    x=dates,
                    y=data_dict['humidity'],
                    name='Humidity',
                    line=dict(color='#06B6D4', width=3),
                    mode='lines+markers'
                ),
                row=2, col=1, secondary_y=False
            )

        if 'pressure' in data_dict:
            fig.add_trace(
                go.Scatter(
                    x=dates,
                    y=data_dict['pressure'],
                    name='Pressure',
                    line=dict(color='#7C3AED', width=3),
                    mode='lines+markers'
                ),
                row=2, col=1, secondary_y=True
            )

        # Wind subplot (polar)
        if 'wind_speed' in data_dict:
            # Check if we have wind_direction data
            if 'wind_direction' in data_dict:
                # Filter out None values
                wind_data = [(s, d) for s, d in zip(data_dict['wind_speed'], data_dict['wind_direction'])
                            if s is not None and d is not None]
            else:
                # If no wind direction, create synthetic directions for visualization
                wind_speeds = [s for s in data_dict['wind_speed'] if s is not None]
                # Generate evenly distributed directions
                if wind_speeds:
                    directions = np.linspace(0, 360, len(wind_speeds), endpoint=False)
                    wind_data = list(zip(wind_speeds, directions))
                else:
                    wind_data = []

            if wind_data:
                speeds, directions = zip(*wind_data)
                fig.add_trace(
                    go.Scatterpolar(
                        r=speeds,
                        theta=directions,
                        mode='markers',
                        name='Wind',
                        marker=dict(
                            size=12,
                            color=speeds,
                            colorscale='Viridis',
                            showscale=True,
                            colorbar=dict(
                                title="Wind Speed<br>(m/s)",
                                titleside="right"
                            )
                        ),
                        text=[f"Speed: {s:.1f} m/s<br>Direction: {d:.0f}°" for s, d in wind_data],
                        hoverinfo='text'
                    ),
                    row=2, col=2
                )
            else:
                # Add a text annotation if no wind data
                fig.add_annotation(
                    text="No wind data available",
                    xref="x4", yref="y4",
                    x=0.5, y=0.5,
                    showarrow=False,
                    row=2, col=2
                )

        # Update layout
        location = data_dict.get('location', data_dict.get('city', 'Unknown Location'))
        period = data_dict.get('period', '')

        main_title = f"<b>Weather Dashboard - {location}</b>"
        if period:
            main_title += f"<br><span style='font-size: 14px; color: #666;'>{period}</span>"

        fig.update_layout(
            title={
                'text': main_title,
                'x': 0.5,
                'xanchor': 'center',
                'font': {'size': 24, 'color': '#1F2937'}
            },
            showlegend=True,
            height=900,
            plot_bgcolor='rgba(255,255,255,0.9)',
            paper_bgcolor='white',
            font=dict(family="Arial, sans-serif", size=12, color="#374151")
        )

        # Update axes
        fig.update_xaxes(title_text="Date", row=1, col=1)
        fig.update_xaxes(title_text="Date", row=1, col=2)
        fig.update_xaxes(title_text="Date", row=2, col=1)

        fig.update_yaxes(title_text="Temperature (°C)", row=1, col=1)
        fig.update_yaxes(title_text="Precipitation (mm)", row=1, col=2)
        fig.update_yaxes(title_text="Humidity (%)", secondary_y=False, row=2, col=1)
        fig.update_yaxes(title_text="Pressure (kPa)", secondary_y=True, row=2, col=1)

        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        filename = f"weather_dashboard_{timestamp}.html"
        fig.write_html(filename)

        return f"Weather dashboard created successfully and saved to {filename}"

    except Exception as e:
        return f"Error creating weather dashboard: {str(e)}"

def create_spi_visualization(data: str) -> str:
    """
    Create Standardized Precipitation Index (SPI) visualization for drought analysis.
    SPI is a normalized index that quantifies precipitation deficit/surplus.
    Modified to match academic paper style with filled areas.
    """
    try:
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots

        import numpy as np
        from datetime import datetime
        from scipy import stats

        # Parse input data
        if isinstance(data, str):
            data_dict = json.loads(data)
        else:
            data_dict = data

        # Extract precipitation data
        precipitation_data = []
        dates = []

        if 'data' in data_dict and isinstance(data_dict['data'], list):
            for day_data in data_dict['data']:
                date_str = day_data.get('date', '')
                precip = day_data.get('precipitation')

                if date_str and precip is not None:
                    dates.append(datetime.strptime(date_str, '%Y-%m-%d'))
                    precipitation_data.append(precip)

        if not precipitation_data:
            return "Error: No precipitation data available for SPI calculation"

        # Group by month-year for SPI calculation
        monthly_precip = {}
        for date, precip in zip(dates, precipitation_data):
            month_key = (date.year, date.month)
            if month_key not in monthly_precip:
                monthly_precip[month_key] = []
            monthly_precip[month_key].append(precip)

        # Calculate monthly totals
        monthly_totals = {}
        monthly_dates = []
        for (year, month), values in monthly_precip.items():
            monthly_totals[(year, month)] = sum(values)
            monthly_dates.append(datetime(year, month, 1))

        # Sort by date
        sorted_months = sorted(monthly_totals.keys())
        monthly_values = [monthly_totals[key] for key in sorted_months]
        sorted_dates = [datetime(year, month, 1) for year, month in sorted_months]

        # Calculate SPI for different time scales
        def calculate_spi(precip_values, window):
            """Calculate SPI for given window (months)"""
            if len(precip_values) < window:
                return [None] * len(precip_values)

            spi_values = []
            for i in range(len(precip_values)):
                if i < window - 1:
                    spi_values.append(None)
                else:
                    # Get window of data
                    window_data = precip_values[max(0, i-window+1):i+1]

                    # Calculate all historical values for this window position
                    all_window_values = []
                    for j in range(window-1, len(precip_values)):
                        historical_window = precip_values[max(0, j-window+1):j+1]
                        if len(historical_window) == window:
                            all_window_values.append(sum(historical_window))

                    if len(all_window_values) > 1:
                        current_sum = sum(window_data)
                        # Fit to normal distribution
                        mean_val = np.mean(all_window_values)
                        std_val = np.std(all_window_values, ddof=1)

                        if std_val > 0:
                            # Calculate SPI (standardized value)
                            spi = (current_sum - mean_val) / std_val
                            spi_values.append(spi)
                        else:
                            spi_values.append(0)
                    else:
                        spi_values.append(None)

            return spi_values

        # Calculate SPI-12 (12-month SPI like in the image)
        spi_12 = calculate_spi(monthly_values, 12)

        # Filter out None values for plotting
        valid_dates = [d for d, s in zip(sorted_dates, spi_12) if s is not None]
        valid_spi = [s for s in spi_12 if s is not None]

        if not valid_spi:
            return "Error: No valid SPI data for visualization"

        # Create the figure
        fig = go.Figure()

        # Separate positive and negative values
        positive_dates = []
        positive_values = []
        negative_dates = []
        negative_values = []
        zero_line_dates = valid_dates
        zero_line_values = [0] * len(valid_dates)

        for date, value in zip(valid_dates, valid_spi):
            if value >= 0:
                positive_dates.append(date)
                positive_values.append(value)
            else:
                negative_dates.append(date)
                negative_values.append(value)

        # Add zero line first (as base)
        fig.add_trace(
            go.Scatter(
                x=zero_line_dates,
                y=zero_line_values,
                mode='lines',
                line=dict(color='black', width=1),
                showlegend=False,
                hoverinfo='skip'
            )
        )

        # Add positive values (wet periods) - filled blue area
        if positive_dates:
            # Create filled area for positive values
            pos_x = []
            pos_y = []
            for date, value in zip(valid_dates, valid_spi):
                pos_x.append(date)
                if value >= 0:
                    pos_y.append(value)
                else:
                    pos_y.append(0)

            fig.add_trace(
                go.Scatter(
                    x=pos_x,
                    y=pos_y,
                    fill='tozeroy',
                    fillcolor='rgb(0, 0, 255)',  # Steel blue matching original image
                    line=dict(color='rgb(0, 0, 255)', width=0.5),
                    mode='lines',
                    name='Wet Periods (SPI > 0)',
                    hovertemplate='SPI: %{y:.2f}<br>Date: %{x}<extra></extra>'
                )
            )

        # Add negative values (dry periods) - filled orange/red area
        if negative_dates:
            # Create filled area for negative values
            neg_x = []
            neg_y = []
            for date, value in zip(valid_dates, valid_spi):
                neg_x.append(date)
                if value < 0:
                    neg_y.append(value)
                else:
                    neg_y.append(0)

            fig.add_trace(
                go.Scatter(
                    x=neg_x,
                    y=neg_y,
                    fill='tozeroy',
                    fillcolor='rgb(253, 0, 30)',  # Chocolate/orange matching original image
                    line=dict(color='rgb(253, 0, 30)', width=0.5),
                    mode='lines',
                    name='Dry Periods (SPI < 0)',
                    hovertemplate='SPI: %{y:.2f}<br>Date: %{x}<extra></extra>'
                )
            )

        # Add horizontal reference lines (more subtle)
        for y_val, style in [(2, 'dot'), (1, 'dash'), (-1, 'dash'), (-2, 'dot')]:
            fig.add_hline(
                y=y_val,
                line_dash=style,
                line_color="gray",
                line_width=0.5,
                opacity=0.6
            )

        # Get location info
        location = data_dict.get('location', 'Unknown Location')
        city = data_dict.get('city', location)

        # If city is still coordinates, try to get real city name
        if '°N' in str(city) and '°E' in str(city):
            try:
                coords = location.replace('°N', '').replace('°E', '').split(', ')
                if len(coords) == 2:
                    lat = float(coords[0])
                    lon = float(coords[1])
                    city_name = get_city_name(lat, lon)
                    if city_name:
                        city = city_name
            except:
                pass

        period = data_dict.get('period', '')

        # Update layout to match academic style
        fig.update_layout(
            title={
                'text': f"<b>Standardized Precipitation Index (SPI-12) - {city}</b>",
                'x': 0.5,
                'xanchor': 'center',
                'font': {'size': 18, 'family': 'Arial, sans-serif', 'color': 'black'}
            },
            height=400,
            width=1000,
            showlegend=True,
            plot_bgcolor='white',
            paper_bgcolor='white',
            font=dict(family="Arial, sans-serif", size=12, color='black'),
            xaxis_title="Years",
            yaxis_title="SPI Value",
            legend=dict(
                orientation="v",
                yanchor="top",
                y=0.98,
                xanchor="left",
                x=0.02,
                bgcolor='rgba(255,255,255,0.8)',
                bordercolor='black',
                borderwidth=1
            ),
            margin=dict(l=80, r=50, t=80, b=80)
        )

        # Configure axes to match academic style
        fig.update_xaxes(
            showgrid=True,
            gridwidth=0.5,
            gridcolor='lightgray',
            showline=True,
            linewidth=1,
            linecolor='black',
            mirror=True,
            tickfont=dict(size=12, color='black')
        )

        fig.update_yaxes(
            showgrid=True,
            gridwidth=0.5,
            gridcolor='lightgray',
            showline=True,
            linewidth=1,
            linecolor='black',
            mirror=True,
            range=[-3.5, 3.5],
            tickfont=dict(size=12, color='black'),
            zeroline=True,
            zerolinewidth=1,
            zerolinecolor='black'
        )

        # Save the figure
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        filename = f"spi_analysis_academic_{timestamp}.html"
        fig.write_html(filename)

        return f"Academic-style SPI analysis visualization created successfully and saved to {filename}"

    except Exception as e:
        return f"Error creating SPI visualization: {str(e)}"



## 3. Data Loading & Initialization

### Mount Google Drive
Access the weather satellite dataset stored in Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Initialize Test Queries
Prepare test queries for system validation.

In [ ]:
test_queries = []

In [ ]:
prompt = """
"""

### System Initialization & Testing

Load a sample satellite image with metadata and test the multi-agent system.

**Test Image**: Morocco satellite imagery  
**Date**: 2023-05-01  
**Purpose**: Validate agent coordination and response quality

In [ ]:
image_path = "/content/drive/MyDrive/wsi-test-data/images/morocco_2023-05-01.png"
metadata_path = "/content/drive/MyDrive/wsi-test-data/metadata/morocco_2023-05-01.json"
# Load metadata from a JSON file
with open(metadata_path, "r") as f:
    metadata = json.load(f)
# Initialize system
weather_system = MultiAgentWeatherSystem()

# Create task with loaded metadata
task = f"analyze satellite image {image_path}|{json.dumps(metadata)}"
result = weather_system.run(task)


### Visualize Agent Graph

Generate a Mermaid diagram showing the agent workflow and connections.

In [ ]:
# Add this to your code to get the exact diagram
system = MultiAgentWeatherSystem()
graph_viz = system.graph.get_graph().draw_mermaid()
print(graph_viz)  # Copy this output to replace the generic mermaid in the PDF

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	router(router)
	weather_agent(weather_agent)
	historical_agent(historical_agent)
	satellite_agent(satellite_agent)
	finalizer(finalizer)
	error_handler(error_handler)
	__end__([<p>__end__</p>]):::last
	__start__ --> router;
	historical_agent --> finalizer;
	router -. &nbsp;error&nbsp; .-> error_handler;
	router -. &nbsp;historical&nbsp; .-> historical_agent;
	router -. &nbsp;satellite&nbsp; .-> satellite_agent;
	router -. &nbsp;weather&nbsp; .-> weather_agent;
	satellite_agent --> finalizer;
	weather_agent --> finalizer;
	error_handler --> __end__;
	finalizer --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



---

## 4. Web Interface Deployment

### Chat Interface
Deploy an interactive web-based chat interface for the multi-agent system.

### Setup Public Access with ngrok

Configure ngrok for public access to the Flask web server.

**What is ngrok?**
- Creates a secure tunnel to your local server
- Provides a public URL for accessing the application
- Useful for demoing applications running on Colab/localhost

In [ ]:
# Setup ngrok and dependencies
ngrokToken = 'YOUR_NGROK_TOKEN'
ngrokTokenCmd = 'ngrok authtoken ' + ngrokToken
!pip -q install pyngrok flask
from pyngrok import ngrok
import subprocess
import shutil

subprocess.run(ngrokTokenCmd.split())

# Create templates directory
!mkdir -p templates
!mkdir -p static/uploads
!mkdir -p static/images

### Copy Static Assets

Copy logo and other static assets to the Flask static directory.

In [ ]:
!cp /content/drive/MyDrive/logo-dgm/logo.jpg static/images/logo.jpg && echo "✅ Logo copied to static/images/logo.jpg"

In [ ]:
shutil.copy('/content/drive/MyDrive/logo-dgm/logo.jpg', 'static/images/logo.jpg')
print("✅ Logo copied to static/images/logo.jpg")

✅ Logo copied to static/images/logo.jpg


### Create Web Interface Template

This cell creates a complete Flask web application with:

#### Frontend Features:
- 🎨 Modern, responsive chat interface
- 📁 File upload support (drag & drop)
- 💬 Real-time message display
- 🖼️ Image preview and display
- 📊 Metadata visualization
- ⚡ Loading indicators

#### Backend Features:
- Flask API endpoints
- Multi-agent system integration
- Image processing
- Session management
- Error handling

#### User Experience:
- Clean, professional design
- Mobile-responsive layout
- Intuitive chat interface
- File upload with preview
- Real-time agent responses

In [ ]:
# Create the enhanced chat HTML template with card-style file upload support
chat_html = """<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>LLM-Powered Agentic AI System</title>
    <link href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/6.4.0/css/all.min.css" rel="stylesheet">
    <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>


    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }

        body {
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, sans-serif;
            background: #f0f0f8;
            color: #333;
            height: 100vh;
            display: flex;
            overflow: hidden;
        }


        .main-chat {
    flex: 1;
    display: flex;
    flex-direction: column;
    background: white;
    width: 100%; /* Add this line */
}


        .chat-header {
            padding: 20px 30px;
            border-bottom: 1px solid #f0f0f0;
            background: white;
            display: flex;
            align-items: center;
            justify-content: space-between;
        }

        .chat-messages {
            flex: 1;
            overflow-y: auto;
            padding: 30px;
            background: #fafafa;
        }

        .message {
            margin-bottom: 30px;
            display: flex;
            align-items: flex-start;
            gap: 15px;
        }

        .message-avatar {
            width: 32px;
            height: 32px;
            border-radius: 50%;
            display: flex;
            align-items: center;
            justify-content: center;
            font-size: 14px;
            flex-shrink: 0;
        }

        .user-avatar {
            background: #333;
            color: white;
        }

        .ai-avatar {
    background: white;
    color: white;
    font-weight: 600;
    border: 2px solid #8b5cf6;
    padding: 0;
    overflow: hidden;
    position: relative;
}

.ai-avatar img {
    width: 100%;
    height: 100%;
    object-fit: cover;
    border-radius: 50%;
}

        .message-content iframe {
            max-width: 100%;
            border-radius: 8px;
        }

        .message-content {
            flex: 1;
            background: white;
            padding: 20px;
            border-radius: 12px;
            border: 1px solid #e5e5ef;
            font-size: 14px;
            line-height: 1.6;
            overflow-x: auto;
        }

        .message-content h3 {
            color: #8b5cf6;
            font-size: 14px;
            margin-bottom: 10px;
            font-weight: 500;
        }

        .message-content ol {
            margin: 15px 0;
            padding-left: 20px;
        }

        .message-content li {
            margin-bottom: 15px;
        }

        .message-content strong {
            color: #333;
            font-weight: 600;
        }

        .message-content pre {
            background: #f8f9fa;
            padding: 10px;
            border-radius: 8px;
            border-left: 3px solid #8b5cf6;
            overflow-x: auto;
            white-space: pre-wrap;
            word-wrap: break-word;
        }

        .message-actions {
            display: flex;
            gap: 15px;
            margin-top: 15px;
            align-items: center;
        }

        .action-btn {
            background: none;
            border: none;
            color: #999;
            cursor: pointer;
            font-size: 16px;
            padding: 5px;
            border-radius: 4px;
            transition: all 0.2s ease;
        }

        .action-btn:hover {
            color: #8b5cf6;
            background: #f8f8ff;
        }

        .regenerate-btn {
            background: none;
            border: none;
            color: #8b5cf6;
            cursor: pointer;
            font-size: 12px;
            font-weight: 500;
            padding: 8px 15px;
            border-radius: 20px;
            border: 1px solid #e5e5ef;
            transition: all 0.2s ease;
            margin-left: auto;
        }

        .regenerate-btn:hover {
            background: #f8f8ff;
            border-color: #8b5cf6;
        }

        .chat-input-container {
            padding: 20px 30px;
            background: white;
            border-top: 1px solid #f0f0f0;
        }

        .file-upload-area {
            display: none;
            margin-bottom: 15px;
            padding: 15px;
            background: #f8f9fa;
            border: 2px dashed #8b5cf6;
            border-radius: 12px;
            text-align: center;
        }

        .file-upload-area.active {
            display: block;
        }

        /* New card-style file display */
        .uploaded-files {
            display: flex;
            flex-wrap: wrap;
            gap: 15px;
            margin-top: 15px;
            padding: 10px 0;
        }

        .file-card {
            background: white;
            border: 2px solid #e5e5ef;
            border-radius: 12px;
            padding: 0;
            width: 200px;
            overflow: hidden;
            position: relative;
            box-shadow: 0 2px 8px rgba(0,0,0,0.1);
            transition: all 0.2s ease;
        }

        .file-card:hover {
            transform: translateY(-2px);
            box-shadow: 0 4px 16px rgba(0,0,0,0.15);
            border-color: #8b5cf6;
        }

        .file-preview {
            width: 100%;
            height: 120px;
            background: #f8f9fa;
            display: flex;
            align-items: center;
            justify-content: center;
            overflow: hidden;
            position: relative;
        }

        .file-preview img {
            width: 100%;
            height: 100%;
            object-fit: cover;
        }

        .file-preview .file-icon {
            font-size: 32px;
            color: #8b5cf6;
        }

        .file-info {
            padding: 12px;
            background: white;
        }

        .file-name {
            font-size: 13px;
            font-weight: 500;
            color: #333;
            margin-bottom: 4px;
            word-break: break-word;
            line-height: 1.3;
        }

        .file-details {
            font-size: 11px;
            color: #666;
            display: flex;
            justify-content: space-between;
            align-items: center;
        }

        .file-type-badge {
            background: #8b5cf6;
            color: white;
            padding: 2px 6px;
            border-radius: 4px;
            font-size: 10px;
            font-weight: 500;
            text-transform: uppercase;
        }

        .file-remove {
            position: absolute;
            top: 8px;
            right: 8px;
            background: rgba(0,0,0,0.6);
            color: white;
            border: none;
            border-radius: 50%;
            width: 24px;
            height: 24px;
            cursor: pointer;
            font-size: 12px;
            display: flex;
            align-items: center;
            justify-content: center;
            transition: all 0.2s ease;
            opacity: 0;
        }

        .file-card:hover .file-remove {
            opacity: 1;
        }

        .file-remove:hover {
            background: #dc2626;
        }

        .chat-input-wrapper {
            position: relative;
            background: #f8f8fa;
            border: 1px solid #e5e5ef;
            border-radius: 25px;
            padding: 12px 100px 12px 50px;
            transition: all 0.2s ease;
        }

        .chat-input-wrapper:focus-within {
            border-color: #8b5cf6;
            background: white;
            box-shadow: 0 0 0 3px rgba(139, 92, 246, 0.1);
        }

        .attach-btn {
            position: absolute;
            left: 15px;
            top: 50%;
            transform: translateY(-50%);
            background: none;
            border: none;
            color: #666;
            cursor: pointer;
            font-size: 18px;
            padding: 5px;
            border-radius: 4px;
            transition: all 0.2s ease;
        }

        .attach-btn:hover {
            color: #8b5cf6;
            background: #f0f0ff;
        }

        .attach-btn.active {
            color: #8b5cf6;
        }

        .chat-input {
            width: 100%;
            background: transparent;
            border: none;
            outline: none;
            font-size: 14px;
            color: #333;
            resize: none;
            min-height: 20px;
            max-height: 120px;
        }

        .chat-input::placeholder {
            color: #999;
        }

        .send-button {
            position: absolute;
            right: 10px;
            top: 50%;
            transform: translateY(-50%);
            background: #8b5cf6;
            border: none;
            color: white;
            width: 36px;
            height: 36px;
            border-radius: 50%;
            cursor: pointer;
            display: flex;
            align-items: center;
            justify-content: center;
            font-size: 14px;
            transition: all 0.2s ease;
        }

        .send-button:hover {
            background: #7c3aed;
            transform: translateY(-50%) scale(1.05);
        }

        .send-button:disabled {
            opacity: 0.5;
            cursor: not-allowed;
            transform: translateY(-50%);
        }

        .typing-indicator {
            display: none;
            align-items: flex-start;
            gap: 15px;
            margin-bottom: 30px;
        }

        .typing-dots {
            background: white;
            padding: 20px;
            border-radius: 12px;
            border: 1px solid #e5e5ef;
            display: flex;
            gap: 4px;
            align-items: center;
        }

        .typing-dot {
            width: 6px;
            height: 6px;
            border-radius: 50%;
            background: #8b5cf6;
            animation: typing 1.4s infinite ease-in-out;
        }

        .typing-dot:nth-child(2) {
            animation-delay: 0.2s;
        }

        .typing-dot:nth-child(3) {
            animation-delay: 0.4s;
        }

        @keyframes typing {
            0%, 60%, 100% {
                transform: translateY(0);
                opacity: 0.3;
            }
            30% {
                transform: translateY(-8px);
                opacity: 1;
            }
        }

        .welcome-screen {
            flex: 1;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            text-align: center;
            color: #666;
            padding: 40px;
        }

        .welcome-screen i {
            font-size: 48px;
            color: #8b5cf6;
            margin-bottom: 20px;
        }

        .welcome-screen h2 {
            color: #333;
            font-size: 24px;
            margin-bottom: 10px;
            font-weight: 600;
        }

        .welcome-screen p {
            font-size: 14px;
            line-height: 1.6;
            max-width: 400px;
        }

        .capabilities {
            display: grid;
            grid-template-columns: repeat(auto-fit, minmax(200px, 1fr));
            gap: 15px;
            margin-top: 30px;
            max-width: 800px;
        }

        .capability {
            background: white;
            border: 1px solid #e5e5ef;
            border-radius: 12px;
            padding: 20px;
            text-align: center;
            transition: all 0.2s ease;
        }

        .capability:hover {
            transform: translateY(-2px);
            box-shadow: 0 4px 15px rgba(139, 92, 246, 0.1);
        }

        .capability i {
            font-size: 24px;
            margin-bottom: 10px;
            color: #8b5cf6;
        }

        .capability h4 {
            color: #333;
            font-size: 14px;
            margin-bottom: 8px;
        }

        .capability p {
            font-size: 12px;
            color: #666;
        }

        .hidden {
            display: none !important;
        }

        /* Scrollbar styling */
        .chat-messages::-webkit-scrollbar {
            width: 6px;
        }

        .chat-messages::-webkit-scrollbar-track {
            background: #f0f0f0;
        }

        .chat-messages::-webkit-scrollbar-thumb {
            background: #d1d1d1;
            border-radius: 3px;
        }

        .chat-messages::-webkit-scrollbar-thumb:hover {
            background: #8b5cf6;
        }


        [data-json] pre {
    font-family: 'Monaco', 'Menlo', 'Ubuntu Mono', monospace;
    font-size: 12px;
    line-height: 1.5;
}

[data-json] code {
    color: #333;
}

    </style>
</head>
<body>
    <div class="main-chat">
        <div class="chat-messages" id="chatMessages">
            <div id="welcomeScreen" class="welcome-screen">
                <i class="fas fa-cloud-sun"></i>
                <h2>LLM-Powered Agentic AI System for Satellite Imagery and Weather Analysis</h2>
                <p>Advanced weather analysis combining satellite image processing, intelligent agent orchestration, and comprehensive meteorological data analytics.</p>
                <div class="capabilities">
                    <div class="capability">
                        <i class="fas fa-satellite"></i>
                        <h4>Satellite Analysis</h4>
                        <p>Upload satellite images for meteorological analysis</p>
                    </div>
                    <div class="capability">
                        <i class="fas fa-chart-bar"></i>
                        <h4>Weather Data</h4>
                        <p>Historical and real-time weather information</p>
                    </div>
                    <div class="capability">
                        <i class="fas fa-file-code"></i>
                        <h4>JSON Metadata</h4>
                        <p>Enhanced analysis with metadata support</p>
                    </div>
                    <div class="capability">
                        <i class="fas fa-brain"></i>
                        <h4>AI-Powered</h4>
                        <p>Intelligent weather pattern recognition</p>
                    </div>
                </div>
            </div>

            <div class="typing-indicator" id="typingIndicator">
                <div class="message-avatar ai-avatar">
                <img src="/static/images/logo.jpg" alt="AI Logo">
            </div>
                <div class="typing-dots">
                    <div class="typing-dot"></div>
                    <div class="typing-dot"></div>
                    <div class="typing-dot"></div>
                </div>
            </div>
        </div>

        <div class="chat-input-container">
            <div class="file-upload-area" id="fileUploadArea">
                <i class="fas fa-cloud-upload-alt" style="font-size: 24px; color: #8b5cf6; margin-bottom: 10px;"></i>
                <p>Drop files here or click to upload</p>
                <p style="font-size: 12px; color: #666; margin-top: 5px;">
                    Supported: Images (JPG, PNG) and JSON files
                </p>
                <input type="file" id="fileInput" multiple accept="image/*,.json" class="hidden">
                <div class="uploaded-files" id="uploadedFiles"></div>
            </div>

            <div class="chat-input-wrapper">
                <button class="attach-btn" id="attachBtn">
                    <i class="fas fa-paperclip"></i>
                </button>
                <textarea
                    id="chatInput"
                    class="chat-input"
                    placeholder="Ask about weather, upload satellite images, or attach JSON metadata..."
                    rows="1"
                ></textarea>
                <button id="sendButton" class="send-button">
                    <i class="fas fa-arrow-up"></i>
                </button>
            </div>
        </div>
    </div>

    <script>
        const chatMessages = document.getElementById('chatMessages');
        const chatInput = document.getElementById('chatInput');
        const sendButton = document.getElementById('sendButton');
        const typingIndicator = document.getElementById('typingIndicator');
        const welcomeScreen = document.getElementById('welcomeScreen');
        const fileUploadArea = document.getElementById('fileUploadArea');
        const fileInput = document.getElementById('fileInput');
        const attachBtn = document.getElementById('attachBtn');
        const uploadedFiles = document.getElementById('uploadedFiles');

        let hasMessages = false;
        let currentFiles = [];

        // Auto-resize textarea
        chatInput.addEventListener('input', function() {
            this.style.height = 'auto';
            this.style.height = Math.min(this.scrollHeight, 120) + 'px';
        });

        // File upload functionality
        attachBtn.addEventListener('click', function() {
            fileUploadArea.classList.toggle('active');
            this.classList.toggle('active');
        });

        fileUploadArea.addEventListener('click', function() {
            fileInput.click();
        });

        fileInput.addEventListener('change', function(e) {
            handleFiles(e.target.files);
        });

        // Drag and drop
        fileUploadArea.addEventListener('dragover', function(e) {
            e.preventDefault();
            this.style.borderColor = '#7c3aed';
        });

        fileUploadArea.addEventListener('dragleave', function(e) {
            e.preventDefault();
            this.style.borderColor = '#8b5cf6';
        });

        fileUploadArea.addEventListener('drop', function(e) {
            e.preventDefault();
            this.style.borderColor = '#8b5cf6';
            handleFiles(e.dataTransfer.files);
        });

        function handleFiles(files) {
            for (let file of files) {
                if (file.type.startsWith('image/') || file.type === 'application/json') {
                    currentFiles.push(file);
                    addFileCard(file);
                }
            }
        }

        function addFileCard(file) {
            const card = document.createElement('div');
            card.className = 'file-card';
            card.dataset.fileName = file.name;

            const preview = document.createElement('div');
            preview.className = 'file-preview';

            const removeBtn = document.createElement('button');
            removeBtn.className = 'file-remove';
            removeBtn.innerHTML = '×';
            removeBtn.onclick = (e) => {
                e.stopPropagation();
                removeFile(file.name);
            };

            const info = document.createElement('div');
            info.className = 'file-info';

            const fileName = document.createElement('div');
            fileName.className = 'file-name';
            fileName.textContent = file.name;

            const details = document.createElement('div');
            details.className = 'file-details';

            const fileSize = document.createElement('span');
            fileSize.textContent = formatFileSize(file.size);

            const typeBadge = document.createElement('span');
            typeBadge.className = 'file-type-badge';

            if (file.type.startsWith('image/')) {
                typeBadge.textContent = file.type.split('/')[1].toUpperCase();

                const reader = new FileReader();
                reader.onload = function(e) {
                    const img = document.createElement('img');
                    img.src = e.target.result;
                    preview.appendChild(img);
                };
                reader.readAsDataURL(file);
            } else if (file.type === 'application/json') {
                typeBadge.textContent = 'JSON';

                const icon = document.createElement('i');
                icon.className = 'fas fa-file-code file-icon';
                preview.appendChild(icon);
            }

            details.appendChild(fileSize);
            details.appendChild(typeBadge);

            info.appendChild(fileName);
            info.appendChild(details);

            card.appendChild(preview);
            card.appendChild(removeBtn);
            card.appendChild(info);

            uploadedFiles.appendChild(card);
        }

        function formatFileSize(bytes) {
            if (bytes === 0) return '0 B';
            const k = 1024;
            const sizes = ['B', 'KB', 'MB', 'GB'];
            const i = Math.floor(Math.log(bytes) / Math.log(k));
            return parseFloat((bytes / Math.pow(k, i)).toFixed(1)) + ' ' + sizes[i];
        }

        function removeFile(fileName) {
            currentFiles = currentFiles.filter(f => f.name !== fileName);
            const fileCard = uploadedFiles.querySelector(`[data-file-name="${fileName}"]`);
            if (fileCard) fileCard.remove();
        }

        // Send message function
        async function sendMessage() {
            const message = chatInput.value.trim();
            if (!message && currentFiles.length === 0) {
                alert("Please type a message or upload a file.");
                return;
            }

            // Hide welcome screen on first message
            if (!hasMessages) {
                welcomeScreen.style.display = 'none';
                hasMessages = true;
            }

            // Add user message
            addMessage(message, 'user', currentFiles.length > 0 ? [...currentFiles] : null);

            const userMessage = message;
            const userFiles = [...currentFiles];

            // Clear input and files
            chatInput.value = '';
            chatInput.style.height = 'auto';
            currentFiles = [];
            uploadedFiles.innerHTML = '';
            fileUploadArea.classList.remove('active');
            attachBtn.classList.remove('active');

            // Show typing indicator
            showTypingIndicator();

            try {
                // Send to backend
                const response = await sendToBackend(userMessage, userFiles);
                hideTypingIndicator();
                addMessage(response, 'ai');
            } catch (error) {
                hideTypingIndicator();
                addMessage(`Error: ${error.message}`, 'ai');
            }
        }

        async function sendToBackend(message, files) {
            const formData = new FormData();
            formData.append('message', message);

            files.forEach(file => {
                formData.append('files', file);
            });

            const response = await fetch('/api/chat', {
                method: 'POST',
                body: formData
            });

            if (!response.ok) {
                throw new Error('Network response was not ok');
            }

            const data = await response.json();
            return data.response || 'No response received';
        }

        // Add message to chat
        function addMessage(text, sender, files = null) {
            const messageDiv = document.createElement('div');
            messageDiv.className = 'message';

            const avatar = document.createElement('div');
            avatar.className = `message-avatar ${sender === 'user' ? 'user-avatar' : 'ai-avatar'}`;
            if (sender === 'user') {
                avatar.textContent = '👤';
            } else {
                avatar.innerHTML = '<img src="/static/images/logo.jpg" alt="AI Logo">';
            }

            const content = document.createElement('div');
            content.className = 'message-content';

            if (sender === 'ai') {
                content.innerHTML = `
                    <h3>Weather Agent</h3>
                    <div>${formatResponse(text)}</div>
                    <div class="message-actions">
                        <button class="action-btn" title="Like">
                            <i class="fas fa-thumbs-up"></i>
                        </button>
                        <button class="action-btn" title="Dislike">
                            <i class="fas fa-thumbs-down"></i>
                        </button>
                        <button class="action-btn" title="Copy">
                            <i class="fas fa-copy"></i>
                        </button>
                    </div>
                `;
            } else {
                let messageContent = text;

                // Add file cards if files exist
                if (files && files.length > 0) {
                    const fileCardsContainer = document.createElement('div');
                    fileCardsContainer.style.cssText = 'margin-top: 15px; margin-bottom: 10px;';

                    const fileLabel = document.createElement('div');
                    fileLabel.style.cssText = 'font-size: 12px; color: #666; margin-bottom: 10px; font-weight: 500;';
                    fileLabel.textContent = 'Files:';
                    fileCardsContainer.appendChild(fileLabel);

                    const cardsGrid = document.createElement('div');
                    cardsGrid.style.cssText = 'display: flex; flex-wrap: wrap; gap: 12px;';

                    files.forEach(file => {
                        const card = createMessageFileCard(file);
                        cardsGrid.appendChild(card);
                    });

                    fileCardsContainer.appendChild(cardsGrid);

                    // Set the content with text and file cards
                    content.innerHTML = messageContent;
                    content.appendChild(fileCardsContainer);
                } else {
                    content.innerHTML = messageContent;
                }
            }

            messageDiv.appendChild(avatar);
            messageDiv.appendChild(content);

            chatMessages.insertBefore(messageDiv, typingIndicator);
            scrollToBottom();
        }

        // Create file card for messages (smaller version)
        function createMessageFileCard(file) {
            const card = document.createElement('div');
            card.style.cssText = `
                background: white;
                border: 1px solid #e5e5ef;
                border-radius: 8px;
                width: 140px;
                overflow: hidden;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                transition: all 0.2s ease;
            `;

            const preview = document.createElement('div');
            preview.style.cssText = `
                width: 100%;
                height: 80px;
                background: #f8f9fa;
                display: flex;
                align-items: center;
                justify-content: center;
                overflow: hidden;
            `;

            const info = document.createElement('div');
            info.style.cssText = 'padding: 8px; background: white;';

            const fileName = document.createElement('div');
            fileName.style.cssText = `
                font-size: 11px;
                font-weight: 500;
                color: #333;
                margin-bottom: 4px;
                word-break: break-word;
                line-height: 1.3;
                overflow: hidden;
                text-overflow: ellipsis;
                white-space: nowrap;
            `;
            fileName.textContent = file.name;
            fileName.title = file.name; // Show full name on hover

            const details = document.createElement('div');
            details.style.cssText = `
                font-size: 10px;
                color: #666;
                display: flex;
                justify-content: space-between;
                align-items: center;
            `;

            const fileSize = document.createElement('span');
            fileSize.textContent = formatFileSize(file.size);

            const typeBadge = document.createElement('span');
            typeBadge.style.cssText = `
                background: #8b5cf6;
                color: white;
                padding: 1px 4px;
                border-radius: 3px;
                font-size: 9px;
                font-weight: 500;
                text-transform: uppercase;
            `;

            if (file.type.startsWith('image/')) {
                typeBadge.textContent = file.type.split('/')[1].toUpperCase();

                const reader = new FileReader();
                reader.onload = function(e) {
                    const img = document.createElement('img');
                    img.src = e.target.result;
                    img.style.cssText = 'width: 100%; height: 100%; object-fit: cover;';
                    preview.appendChild(img);
                };
                reader.readAsDataURL(file);
            } else if (file.type === 'application/json') {
                typeBadge.textContent = 'JSON';

                const icon = document.createElement('i');
                icon.className = 'fas fa-file-code';
                icon.style.cssText = 'font-size: 24px; color: #8b5cf6;';
                preview.appendChild(icon);
            }

            details.appendChild(fileSize);
            details.appendChild(typeBadge);

            info.appendChild(fileName);
            info.appendChild(details);

            card.appendChild(preview);
            card.appendChild(info);

            return card;
        }

        function formatResponse(text) {
    // Check if response contains JSON file reference
    const jsonFileMatch = text.match(/([a-zA-Z_]+_\d{14}\.json)/);
    if (jsonFileMatch) {
        const filename = jsonFileMatch[1];
        const beforeFile = text.substring(0, text.indexOf(filename));
        const afterFile = text.substring(text.indexOf(filename) + filename.length);

        // Fetch and display JSON content
        fetch(`/files/${filename}`)
            .then(response => response.json())
            .then(data => {
                const jsonDisplay = document.querySelector(`[data-json="${filename}"]`);
                if (jsonDisplay) {
                    jsonDisplay.innerHTML = `
                        <div style="background: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; overflow: hidden; margin: 15px 0;">
                            <div style="background: #e9ecef; padding: 12px; border-bottom: 1px solid #dee2e6; font-weight: 600; color: #495057; display: flex; align-items: center; gap: 8px;">
                                <i class="fas fa-file-code"></i>
                                Historical Weather Data - ${data.city || data.location || 'Unknown Location'}
                                <span style="margin-left: auto; font-size: 12px; font-weight: normal;">${data.days_count || 0} days</span>
                            </div>
                            <pre style="margin: 0; padding: 20px; background: #f8f9fa; color: #212529; font-size: 13px; line-height: 1.4; max-height: 400px; overflow-y: auto;"><code>${JSON.stringify(data, null, 2)}</code></pre>
                        </div>
                    `;
                }
            })
            .catch(error => {
                console.error('Error loading JSON:', error);
                const jsonDisplay = document.querySelector(`[data-json="${filename}"]`);
                if (jsonDisplay) {
                    jsonDisplay.innerHTML = `
                        <div style="background: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 20px; margin: 15px 0; text-align: center; color: #dc2626;">
                            <i class="fas fa-exclamation-triangle"></i> Error loading data file
                        </div>
                    `;
                }
            });

        return `${marked.parse(beforeFile)}<br>
                <div data-json="${filename}">
                    <div style="background: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 20px; margin: 15px 0; text-align: center;">
                        <i class="fas fa-spinner fa-spin"></i> Loading historical weather data...
                    </div>
                </div>
                ${marked.parse(afterFile)}`;
    }

    // Check if response contains HTML file reference
    const htmlFileMatch = text.match(/([a-zA-Z_]+_\d{14}\.html)/);
    if (htmlFileMatch) {
        const filename = htmlFileMatch[1];
        const beforeFile = text.substring(0, text.indexOf(filename));
        const afterFile = text.substring(text.indexOf(filename) + filename.length);

        return `${marked.parse(beforeFile)}<br><br>
                <div style="border: 1px solid #e5e5ef; border-radius: 12px; overflow: hidden; margin: 15px 0;">
                    <div style="background: #f8f9fa; padding: 10px; border-bottom: 1px solid #e5e5ef; font-weight: 500; color: #333;">
                        📊 Generated Visualization
                    </div>
                    <iframe src="/files/${filename}"
                            style="width: 100%; height: 500px; border: none;"
                            frameborder="0">
                    </iframe>
                </div>
                ${marked.parse(afterFile)}`;
    }

    // Format JSON responses
    try {
        const jsonData = JSON.parse(text);
        return `<pre>${JSON.stringify(jsonData, null, 2)}</pre>`;
    } catch (e) {
        // Use `marked` for Markdown rendering
        return marked.parse(text);
    }
}
        // Show/hide typing indicator
        function showTypingIndicator() {
            typingIndicator.style.display = 'flex';
            scrollToBottom();
        }

        function hideTypingIndicator() {
            typingIndicator.style.display = 'none';
        }

        // Scroll to bottom
        function scrollToBottom() {
            chatMessages.scrollTop = chatMessages.scrollHeight;
        }





        // Event listeners
        sendButton.addEventListener('click', sendMessage);

        chatInput.addEventListener('keydown', function(e) {
            if (e.key === 'Enter' && !e.shiftKey) {
                e.preventDefault();
                sendMessage();
            }
        });

        // Focus on input when page loads
        window.addEventListener('load', function() {
            chatInput.focus();
        });

        // Conversation item click handlers
        document.querySelectorAll('.conversation-item').forEach(item => {
            item.addEventListener('click', function() {
                document.querySelectorAll('.conversation-item').forEach(i => i.classList.remove('active'));
                this.classList.add('active');
            });
        });
    </script>
</body>
</html>"""

# Write the HTML file
with open('templates/chat.html', 'w', encoding='utf-8') as f:
    f.write(chat_html)

# Enhanced Flask application with Weather Agent integration
from flask import Flask, render_template, request, jsonify
import os
import json
import base64
from werkzeug.utils import secure_filename
import threading
import time

# PASTE YOUR WEATHER AGENT CODE HERE (all the classes and functions from your document)
# Just paste the entire content of your agent code here without modifications

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'static/uploads'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB max file size

# Initialize the weather system (this will happen after you paste your agent code)
try:
    weather_system = MultiAgentWeatherSystem()
    print("✅ Weather Agent System initialized successfully!")
except Exception as e:
    print(f"❌ Error initializing weather system: {e}")
    weather_system = None

@app.route("/")
def hello():
    return 'CHAT A.I+ Weather Agent is running! Visit /chat to start chatting.'

@app.route('/chat')
def chat():
    return render_template('chat.html')

@app.route('/api/chat', methods=['POST'])
def chat_api():
    """Enhanced API endpoint for chat messages with file support"""
    try:
        user_message = request.form.get('message', '')
        uploaded_files = request.files.getlist('files')

        # Process uploaded files
        file_info = []
        image_path = None
        json_metadata = None

        for file in uploaded_files:
            if file and file.filename:
                filename = secure_filename(file.filename)
                filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)

                # Create upload directory if it doesn't exist
                os.makedirs(os.path.dirname(filepath), exist_ok=True)

                file.save(filepath)
                file_info.append(f"Uploaded: {filename}")

                # Check file type
                if file.content_type and file.content_type.startswith('image/'):
                    image_path = filepath
                elif filename.endswith('.json'):
                    try:
                        with open(filepath, 'r') as f:
                            json_metadata = json.load(f)
                    except Exception as e:
                        file_info.append(f"Error reading JSON {filename}: {str(e)}")

        # Prepare the query for the weather agent
        if image_path and json_metadata:
            # Image + JSON metadata case
            agent_query = f"analyze satellite image {image_path} with metadata {json.dumps(json_metadata)}"
        elif image_path:
            # Image only case
            agent_query = f"analyze satellite image {image_path}"
        elif user_message:
            # Text only case
            agent_query = user_message
        else:
            agent_query = "Hello, how can I help you with weather analysis?"

        # Add file info to user message for display
        if file_info:
            user_message += f"\n\nFiles processed: {', '.join(file_info)}"

        # Get response from weather agent
        if weather_system:
            try:
                result = weather_system.run(agent_query)
                ai_response = result.get('output', 'No response generated')

                # Add file processing info if any
                if file_info:
                    ai_response = f"**Files processed:** {', '.join(file_info)}\n\n{ai_response}"

            except Exception as e:
                ai_response = f"Weather Agent Error: {str(e)}"
        else:
            ai_response = "Weather agent not initialized. Please check system setup."

        return jsonify({
            'response': ai_response,
            'status': 'success',
            'files_processed': len(uploaded_files)
        })

    except Exception as e:
        return jsonify({
            'response': f"Error processing request: {str(e)}",
            'status': 'error'
        }), 500

@app.route('/files/<filename>')
def serve_file(filename):
    """Serve generated HTML files"""
    try:
        file_path = os.path.join(os.getcwd(), filename)
        if os.path.exists(file_path):
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
            return content
        else:
            return "File not found", 404
    except Exception as e:
        return f"Error: {str(e)}", 500

# Start the app with ngrok
if __name__ == "__main__":
    print("Setting up ngrok tunnel...")
    public_url = ngrok.connect(5001)
    print(f"🚀 CHAT A.I+ Weather Agent is live at: {public_url}")
    print(f"📱 Visit: {public_url}/chat to start chatting!")
    print(f"📁 Upload folder: {app.config['UPLOAD_FOLDER']}")

    # Run Flask app
    app.run(host="0.0.0.0", port=5001, debug=False)

<>:984: SyntaxWarning: invalid escape sequence '\d'
<>:984: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipython-input-626718563.py:984: SyntaxWarning: invalid escape sequence '\d'
  const jsonFileMatch = text.match(/([a-zA-Z_]+_\d{14}\.json)/);


✅ Weather Agent System initialized successfully!
Setting up ngrok tunnel...
🚀 CHAT A.I+ Weather Agent is live at: NgrokTunnel: "https://5bc8f666b986.ngrok-free.app" -> "http://localhost:5001"
📱 Visit: NgrokTunnel: "https://5bc8f666b986.ngrok-free.app" -> "http://localhost:5001"/chat to start chatting!
📁 Upload folder: static/uploads
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://172.28.0.12:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Sep/2025 16:55:51] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2025 16:55:52] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2025 16:55:56] "GET /chat HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2025 16:55:57] "GET /static/images/logo.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Sep/2025 16:56:00] "GET /static/images/logo.jpg HTTP/1.1" 200 -


[Enhanced System] Starting with task: analyze satellite image static/uploads/morocco_2023-05-01.png with metadata {"image_info": {"capture_date": "2023-05-01", "capture_time_utc": "12:00 UTC", "satellite": "MODIS Terra", "product": "CorrectedReflectance_TrueColor", "resolution": "250m"}, "temporal_synchronization": {"satellite_acquisition_time": "12:00 UTC", "weather_data_time": "12:00 UTC", "synchronization_quality": "Perfect"}, "geographic_info": {"country": "Morocco", "bounding_box": {"west": -17.5, "south": 21.0, "east": 0.0, "north": 36.0}, "regions_monitored": ["casablanca", "rabat", "marrakesh", "fes", "tangier", "agadir", "atlas_mountains", "sahara_south", "rif_mountains", "oujda"]}, "temporal_info": {"year": 2023, "month": 5, "season": "Spring", "day_of_week": "Monday"}, "regional_weather": {"casablanca": {"location": "Economic capital and largest city, Atlantic coast", "coordinates": {"lat": 33.5731, "lon": -7.5898}, "weather_12utc": {"T2M": 27.5, "RH2M": 32.83, "WS10M": 3.76

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

...

Yesterday's weather in Rabat, Morocco (2025-08-31) was as follows:

- **Condition:** Sunny
- **Temperature:** 22.6°C
- **Max Temperature:** 25.0°C
- **Min Temperature:** 20.7°C
- **Precipitation:** 0.1 mm
- **Humidity:** 77%
- **Wind Speed:** 5.9 km/h

This data was retrieved using WeatherAPI.

> Finished chain.

[Finalizer] Task completed successfully
[Enhanced System] Starting with task: What's the current weather in Rabat?

[Enhanced Router] Routing task to weather agent
[Enhanced Router] Task type detected: what's the current weather in rabat?


> Entering new AgentExecutor chain...

Invoking: `get_current_weather` with `Rabat`


{
  "date": "2025-09-01",
  "location": "Rabat, Morocco",
  "temperature": 23.3,
  "temperature_max": null,
  "temperature_min": null,
  "feels_like": 25.0,
  "precipitation": 0.0,
  "humidity": 57,
  "wind_speed": 6.111111111111111,
  "wind_direction": 3,
  "pressure": 101.7,
  "condition": "Sunny",
  "api_used": "WeatherAPI"
}

INFO:werkzeug:127.0.0.1 - - [01/Sep/2025 17:05:32] "POST /api/chat HTTP/1.1" 200 -


The current weather in Rabat, Morocco on September 1, 2025, is as follows:

- **Condition**: Sunny
- **Temperature**: 23.3°C
- **Feels Like**: 25.0°C
- **Precipitation**: 0.0 mm
- **Humidity**: 57%
- **Wind Speed**: 6.1 km/h
- **Wind Direction**: 3° (North)
- **Pressure**: 101.7 kPa

Enjoy the sunny day!

> Finished chain.

[Finalizer] Task completed successfully
[Enhanced System] Starting with task: What was the weather like in Beni mellal 3 days ago?

[Enhanced Router] Routing task to weather agent
[Enhanced Router] Task type detected: what was the weather like in beni mellal 3 days ago?


> Entering new AgentExecutor chain...

Invoking: `get_intelligent_weather` with `Beni Mellal,2025-08-29`


{
  "date": "2025-08-29",
  "location": "Beni Mellal, Morocco",
  "temperature": 22.8,
  "temperature_max": 28.6,
  "temperature_min": 18.0,
  "precipitation": 0.0,
  "humidity": 37,
  "wind_speed": 4.5,
  "condition": "Sunny",
  "api_used": "WeatherAPI"
}

INFO:werkzeug:127.0.0.1 - - [01/Sep/2025 17:05:54] "POST /api/chat HTTP/1.1" 200 -


The weather in Beni Mellal, Morocco on August 29, 2025, was sunny. The temperature was around 22.8°C, with a maximum of 28.6°C and a minimum of 18.0°C. There was no precipitation, the humidity was at 37%, and the wind speed was 4.5 km/h.

> Finished chain.

[Finalizer] Task completed successfully


## 5. Load Fine-Tuned Vision Model

### LLaVA-OneVision Model Loading

Load the fine-tuned LLaVA-OneVision model for satellite image analysis.

**Model**: `llava-onevision-qwen2-7b-ov-hf`  
**Precision**: bfloat16  
**Device**: Auto (GPU if available)  

This model has been fine-tuned specifically for weather satellite imagery analysis.

In [ ]:
model = LlavaOnevisionForConditionalGeneration.from_pretrained(
                "llava-hf/llava-onevision-qwen2-7b-ov-hf",
                torch_dtype=torch.bfloat16,
                device_map="auto",
                trust_remote_code=True
            )


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.23G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/126 [00:00<?, ?B/s]

---


### How to Use

1. **Run Setup Cells**: Execute cells 1-3 to install dependencies and configure API
2. **Initialize System**: Run the agent system cell (main class)
4. **Test System**: Run test queries to validate agents
5. **Deploy Web Interface**: Setup ngrok and Flask server
6. **Access Application**: Open the ngrok URL in your browser
7. **Interact**: Upload satellite images and ask questions





**Related Notebooks:**
- [LLaVA Fine-Tuning](../LLaVA_finetuning.ipynb)
- [Qwen2-VL Fine-Tuning](../fine_tuning-qwen.ipynb)

---


**Author**: azeddin sahir  
**Application**: Multi-Agent Weather Analysis System  
